In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io as sio



# GT and phantom loading

# Mask path

In [ ]:
mask_path = "/Dinor_revision/new_phantom/new_phantom_data/80_by_80_segmented_3D_masks_test/scan12.h5"
mask_h5 = h5py.File(mask_path,'r')
print(mask_h5.keys())
mask = mask_h5['masks'][:]
mask = mask.astype(np.float32)  # or np.float64
mask[mask == 0] = np.nan
print(mask.shape)



In [ ]:
mask_3D = "/home/sahar/Models/phantom_data/new_B0_data/masks_80/scan12_masks.h5"
mask_3D = h5py.File(mask_3D,'r')
print(mask_3D.keys())
mask_3D = mask_3D['mask'][:]
mask_3D = np.transpose(mask_3D,(2,0,1))
mask_3D = mask_3D.astype(np.float32)
mask_3D[mask_3D == 0] = np.nan
print(mask_3D.shape)
plt.imshow(mask_3D[5])


create a 4D GT dataset

In [ ]:
# Bpath
b_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/scan12_B_pred.h5"
#"/home/sahar/Models/phantom_data/new_B0_data/dataset_80/B_maps.h5"
b = h5py.File(b_path, 'r')
keys = list(b.keys())
data_1 = b[keys[-1]][:]
b_maps = np.transpose(data_1, (0,3,1,2))
b0_gt = b_maps[0,:,:,:] * mask
b1_gt = b_maps[1,:,:,:] * mask
b1_gt_display = b_maps[1,:,:,:] * mask_3D
b0_gt_display = b_maps[0,:,:,:] * mask_3D
print(b0_gt.shape, b1_gt.shape)
print(b_maps.shape)
print(data_1.shape)
# Kswap axes
ksw_path = "/Dinor_revision/new_phantom/new_phantom_data/maps_80/ksw_maps_3D_test/scan12.h5"
ksw_h5 = h5py.File(ksw_path,'r')
keys = list(ksw_h5.keys())
ksw_gt = ksw_h5[keys[-1]][:] * mask
print(ksw_gt.shape)
# fs masp
fs_path = "/Dinor_revision/new_phantom/new_phantom_data/maps_80/fs_maps_3D_test/scan12.h5"
fs_h5 = h5py.File(fs_path,'r')
keys = list(fs_h5.keys())
fs_gt = fs_h5[keys[-1]][:] * mask
print(fs_gt.shape)
# T1 path
t1_path = "/Dinor_revision/new_phantom/new_phantom_data/maps_80/T1_maps_3D_test/scan12.h5"
t1_h5 = h5py.File(t1_path,'r')
keys = list(t1_h5.keys())
t1_gt = t1_h5[keys[-1]][:] * mask
print(t1_gt.shape)
# T2 path
t2_path = "/Dinor_revision/new_phantom/new_phantom_data/maps_80/T2_maps_3D_test/scan12.h5"
t2_h5 = h5py.File(t2_path,'r')
keys = list(t2_h5.keys())
t2_gt = t2_h5[keys[-1]][:] * mask
print(t2_gt.shape)

fig, ax = plt.subplots(1,6, figsize=(20,5))
ax[0].imshow(ksw_gt[10,:,:], cmap='viridis')

ax[1].imshow(fs_gt[10,:,:], cmap='viridis')
ax[2].imshow(b0_gt[10,:,:], cmap='viridis')
ax[3].imshow(b1_gt[10,:,:], cmap='viridis')
ax[4].imshow(t1_gt[10,:,:], cmap='viridis')
ax[5].imshow(t2_gt[10,:,:], cmap='viridis')

gt = np.stack((ksw_gt, fs_gt, b0_gt, b1_gt, t1_gt, t2_gt), axis=0)
gt_disp = np.stack((ksw_gt, fs_gt, b0_gt_display, b1_gt_display, t1_gt, t2_gt), axis=0)
print(gt.shape)

with h5py.File('/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/scan12_gt.h5', 'w') as f:
    f.create_dataset('gt', data=gt, compression="gzip")
    f.create_dataset('gt_disp', data=gt_disp, compression="gzip")

for i in range(6):
    print(f"Parameter {i} max value gt {np.nanmax(gt[i,:,:,:])}")


In [ ]:
# Pred ksw, fs
path = "/comp/fs_ksw_out/y_pred_4D_masked_scaled.h5"
data = h5py.File(path,'r')
print(data.keys())
data = data['predictions'][:]
fs = data[:12,0,:,:]
fs[fs < 1] = np.nan
ksw = data[:12,1,:,:]
ksw[ksw < 1] = np.nan
print(f"fs shape: {fs.shape}, ksw shape: {ksw.shape}")

# Gt ksw and fs
fs_gt_path = "/comp/fs_maps_3D_test/scan12.h5"
fs_gt_h5 = h5py.File(fs_gt_path,'r')
keys = list(fs_gt_h5.keys())
fs_gt = fs_gt_h5[keys[-1]][:]
fs_gt = fs_gt[:12,:,:]
fs_gt[fs_gt == 0] = np.nan

ksw_gt_path = "/comp/ksw_maps_3D_test/scan12.h5"
ksw_gt_h5 = h5py.File(ksw_gt_path,'r')
keys = list(ksw_gt_h5.keys())
ksw_gt = ksw_gt_h5[keys[-1]][:]
ksw_gt = ksw_gt[:12,:,:]
ksw_gt[ksw_gt == 0] = np.nan
print(f"ksw gt shape {ksw_gt.shape}, fs gt shape: {fs_gt.shape} ")
# Pred T
path_T = "/comp/T1_out/y_pred_4D_T1_T2_scaled.h5"
data_T = h5py.File(path_T,'r')
print(data_T.keys())
data_T = data_T['predictions'][:]
print(data_T.shape)
t1 = data_T[:12,0,:,:]
t1[t1 < 2] = np.nan
t2 = data_T[:12,1,:,:]
t2[t2 < 2] = np.nan
print(f"T1 shape: {t1.shape}, T2 shape: {t2.shape}")

# GT T1 and T2 for tbmf
target_T = "/home/sahar/Models/comp/T1_out/y_targets_4D_T1_T2_scaled.h5"
t_data = h5py.File(target_T,'r')
print(t_data.keys())
t_data = t_data['targets'][:]
print(t_data.shape)
t1_gt = t_data[:12,0,:,:]* mask_3D
t1_gt[t1_gt == 0] = np.nan
t2_gt = t_data[:12,1,:,:]* mask_3D
t2_gt[t2_gt == 0] = np.nan
print(f"T1_gt shape: {t1_gt.shape}, T2_gt shape: {t2_gt.shape}")



path_B = "/comp/B_out/y_pred_4D_B0_B1_scaled.h5"
data_B = h5py.File(path_B,'r')
print(data_B.keys())
data_B = data_B['predictions'][:]
print(data_B.shape)
b0 = data_B[:12,0,:,:] * mask
b0_display = data_B[:12,0,:,:] * mask_3D
b1 = data_B[:12,1,:,:] * mask
b1_display = data_B[:12,1,:,:] * mask_3D
print(f"B0 shape: {b0.shape}, B1 shape: {b1.shape}")

tbmf = np.stack((ksw, fs, b0, b1, t1, t2), axis=0)
tbmf_display = np.stack((ksw, fs,b0_display, b1_display, t1, t2), axis=0)
tbmf_gt = np.stack((ksw_gt,fs_gt, b0_gt, b1_gt, t1_gt, t2_gt), axis=0)
print(f"tbmf shape: {tbmf.shape}, tbmf_gt.shape = {tbmf_gt.shape}")

with h5py.File("/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/tbmf.h5",'w') as f:
    f.create_dataset("tmbf", data=tbmf, compression="gzip")
    f.create_dataset("tbmf_gt", data=tbmf_gt, compression="gzip")
#
fig, ax = plt.subplots(3,6, figsize=(20,5))
for i in range(6):
    ax[0,i].imshow(tbmf[i,10,:,:] , cmap='viridis')
    ax[1,i].imshow(tbmf_gt[i,10,:,:], cmap='viridis')
    ax[2,i].imshow(tbmf_display[i,10,:,:], cmap='viridis')
    print(f"Parameter {i} max value pred {np.nanmax(tbmf[i,:,:,:])}, gt- {np.nanmax(tbmf_gt[i,:,:,:])}")


In [ ]:
import scipy.ndimage as ndimage
from scipy.ndimage import center_of_mass, shift


def rotate_image(image, axes=(1,2), k =1):
    # option to rotate the image along different axes with variaty of k
    return ndimage.rotate(image, angle=90*k, axes=axes, reshape=False, order=0)



# Config

In [ ]:
config = {
    "save_path" :"/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp",
    "nikita_image_size" : (6,80,12,80),
    "alex_image_size" : (2,80,12,80)

}

# Nikitas methods:

In [ ]:

# Nikita 6:
nikita6_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/nikita/phantoms/phantoms_demo6/qmaps.mat"
nikita6 = sio.loadmat(nikita6_path)
print(nikita6.keys())
maps = ['ksw','fs','b0_inhom','','t1w','t2w']
y_nikita6 = np.zeros((6,80,12,80))

for i,map  in enumerate(maps):
    if map == '':
        continue
    y_nikita6[i,:,:,:] = nikita6[map]
    print(f"for map {map}, min: {np.min(y_nikita6[i,:,:,:])}, max: {np.max(y_nikita6[i,:,:,:])}")

y_nikita6 = np.transpose(y_nikita6, (0,2,1,3))
y_nikita_6_masked = np.zeros(y_nikita6.shape)
y_nikita_6_display = np.zeros(y_nikita6.shape)
print(f"y nikita6 shape: {y_nikita6.shape}, ymasked sahpe: {y_nikita_6_masked.shape}")
#
for i in range(y_nikita6.shape[0]):
    y_nikita_6_masked[i] = y_nikita6[i] * mask[:,:,:]
    if i ==2:
        y_nikita_6_display[i,:,:,:] = y_nikita6[i] * mask_3D
    else:
        y_nikita_6_display[i,:,:,:] = y_nikita6[i,:,:,:] * mask[:,:,:]

fig, ax = plt.subplots(2,6, figsize=(20,5))
for i in range(6):
    ax[0,i].imshow(y_nikita_6_masked[i,10,:,:] , cmap='viridis')
    ax[1,i].imshow(y_nikita6[i,10,:,:], cmap='viridis')

with h5py.File(os.path.join(config['save_path'], f'nikita_6.h5'), 'w') as f:
    f.create_dataset('y_pred', data=y_nikita6)
    f.create_dataset('y_pred_masked', data=y_nikita_6_masked)

print(y_nikita6.shape)

# Nikita 30 :
nikita30_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/nikita/phantoms/phantoms_demo/qmaps.mat"
nikita30 = sio.loadmat(nikita30_path)
print(nikita30.keys())
y_nikita30 = np.zeros(config['nikita_image_size'])

for i,map  in enumerate(maps):
    if map == '':
        continue
    y_nikita30[i,:,:,:] = nikita30[map]
    print(f"for map {map}, min: {np.min(y_nikita30[i,:,:,:])}, max: {np.max(y_nikita30[i,:,:,:])}")


y_nikita30 = np.transpose(y_nikita30, (0,2,1,3))
y_nikita_30_masked = np.zeros(y_nikita30.shape)
y_nikita_30_display = np.zeros(y_nikita30.shape)

for i in range(y_nikita30.shape[0]):
    y_nikita_30_masked[i] = y_nikita30[i] * mask
    if i==2:
        y_nikita_30_display[i,:,:,:] = y_nikita30[i] * mask_3D
    else:
        y_nikita_30_display[i,:,:,:] = y_nikita30[i,:,:,:] * mask

with h5py.File(os.path.join(config['save_path'], f'nikita_30.h5'), 'w') as f:
    f.create_dataset('y_pred', data=y_nikita30)
    f.create_dataset('y_pred_masked', data=y_nikita_30_masked)
print(y_nikita6.shape)

fig, ax = plt.subplots(3,6, figsize=(20,8))
for i in range(6):
    im0 = ax[0,i].imshow(y_nikita_30_masked[i,10,:,:], cmap='viridis')
    im1 = ax[1,i].imshow(y_nikita30[i,10,:,:], cmap='viridis')
    im2 = ax[2,i].imshow(gt[i,10,:,:], cmap="viridis")

    # Add colorbars
    plt.colorbar(im0, ax=ax[0,i], shrink=0.6)
    plt.colorbar(im1, ax=ax[1,i], shrink=0.6)
    plt.colorbar(im2, ax=ax[2,i], shrink=0.6)

    # Optional: Add titles for clarity
    if i < len(maps) and maps[i] != '':
        ax[0,i].set_title(f'{maps[i]} (masked)')
        ax[1,i].set_title(f'{maps[i]} (unmasked)')
        ax[2,i].set_title(f'{maps[i]} (ground truth)')

plt.tight_layout()


#Alex dataset

In [ ]:
# Alex nbmf 6:
maps_alex = ['kb_T', 'fb_T']
alex_nbmf_6_path = f"/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/alex/nbmf_phantom_6.h5"
alex_nbmf_6_file = h5py.File(alex_nbmf_6_path, 'r')

print(alex_nbmf_6_file.keys())
y_alex_nbmf_6 = np.zeros(config['alex_image_size'])

for i,map  in enumerate(maps_alex):
    # handle Nan values if any
    if np.any(np.isnan(alex_nbmf_6_file[map][:])):
        print(f"Warning: NaN values found in map {map}. Replacing NaNs with zeros.")
        y_alex_nbmf_6[i,:,:,:] = np.nan_to_num(alex_nbmf_6_file[map][:], nan=0.0)
    else:
        y_alex_nbmf_6[i,:,:,:] = alex_nbmf_6_file[map][:]
    print(f"for map {map}, min: {np.min(y_alex_nbmf_6[i,:,:,:])}, max: {np.max(y_alex_nbmf_6[i,:,:,:])}")

y_alex_nbmf_6 = np.transpose(y_alex_nbmf_6, (0,2,1,3))
y_alex_nbmf_6_masked= np.zeros(y_alex_nbmf_6.shape)

for i in range(y_alex_nbmf_6.shape[0]):
    y_alex_nbmf_6_masked[i,:,:,:] = y_alex_nbmf_6[i,:,:,:] * mask[:,:,:]


with h5py.File(os.path.join(config['save_path'], f'alex_nbmf_6.h5'), 'w') as f:
    f.create_dataset('y_pred', data=y_alex_nbmf_6)
    f.create_dataset('y_pred_masked', data=y_alex_nbmf_6_masked)

print(f" shape of nbmf :{y_alex_nbmf_6.shape}")
#
# fig, ax = plt.subplots(2,2, figsize=(10,5))
# for i in range(2):
#     ax[0,i].imshow(y_alex_nbmf_6[i,10,:,:] , cmap='viridis')
#     ax[1,i].imshow(y_alex_nbmf_6_masked[i,10,:,:]*110e3/3, cmap='viridis')


# Alex nbmf 30:
alex_nbmf_30_path = f"/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/alex/nbmf_phantom_30.h5"
alex_nbmf_30_file = h5py.File(alex_nbmf_30_path, 'r')
print(alex_nbmf_30_file.keys())
y_alex_nbmf_30 = np.zeros(config['alex_image_size'])

for i,map  in enumerate(maps_alex):
    if np.any(np.isnan(alex_nbmf_30_file[map][:])):
        print(f"Warning: NaN values found in map {map}. Replacing NaNs with zeros.")
        y_alex_nbmf_30[i,:,:,:] = np.nan_to_num(alex_nbmf_30_file[map][:], nan=0.0)
    else:
        y_alex_nbmf_30[i,:,:,:] = alex_nbmf_30_file[map][:]
    print(f"for map {map}, min: {np.min(y_alex_nbmf_30[i,:,:,:])}, max: {np.max(y_alex_nbmf_30[i,:,:,:])}")

y_alex_nbmf_30 = np.transpose(y_alex_nbmf_30, (0,2,1,3))
y_alex_nbmf_30_masked = np.zeros(y_alex_nbmf_30.shape)

for i in range(y_alex_nbmf_30.shape[0]):
    y_alex_nbmf_30_masked[i,:,:,:] = y_alex_nbmf_30[i,:,:,:] * mask[:,:,:]

with h5py.File(os.path.join(config['save_path'], f'alex_nbmf_30.h5'), 'w') as f:
    f.create_dataset('y_pred', data=y_alex_nbmf_30)
    f.create_dataset('y_pred_masked', data=y_alex_nbmf_30_masked)


# fig, ax = plt.subplots(3,2, figsize=(10,5))
# for i in range(2):
#     im0 = ax[0,i].imshow(y_alex_nbmf_30_masked[i,10,:,:], cmap='viridis')
#     im1 = ax[1,i].imshow(y_alex_nbmf_30[i,10,:,:], cmap='viridis')
#     im2 = ax[2,i].imshow(gt[i,10,:,:], cmap="viridis")
#
#     # Add colorbars
#     plt.colorbar(im0, ax=ax[0,i], shrink=0.6)
#     plt.colorbar(im1, ax=ax[1,i], shrink=0.6)
#     plt.colorbar(im2, ax=ax[2,i], shrink=0.6)
#
#     # Optional: Add titles for clarity
#     if i < len(maps) and maps[i] != '':
#         ax[0,i].set_title(f'{maps[i]} (masked)')
#         ax[1,i].set_title(f'{maps[i]} (unmasked)')
#         ax[2,i].set_title(f'{maps[i]} (ground truth)')
#
# plt.tight_layout()
# print(y_alex_nbmf_30.shape)
# #
#
# Alex vbmf 6:
alex_vbmf_6_path = f"/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/alex/vbmf_phantom-larg2_6.h5"
alex_vbmf_6_file = h5py.File(alex_vbmf_6_path, 'r')
print(alex_vbmf_6_file.keys())

y_alex_vbmf_6 = np.zeros(config['alex_image_size'])
for i,map  in enumerate(maps_alex):
    if np.any(np.isnan(alex_vbmf_6_file[map][:])):
        print(f"Warning: NaN values found in map {map}. Replacing NaNs with zeros.")
        y_alex_vbmf_6[i,:,:,:] = np.nan_to_num(alex_vbmf_6_file[map][:], nan=0.0)
    else:
        y_alex_vbmf_6[i,:,:,:] = alex_vbmf_6_file[map][:]
    print(f"for map {map}, min: {np.min(y_alex_vbmf_6[i,:,:,:])}, max: {np.max(y_alex_vbmf_6[i,:,:,:])}")

y_alex_vbmf_6 = np.transpose(y_alex_vbmf_6, (0,2,1,3))
y_alex_vbmf_6_masked = np.zeros(y_alex_vbmf_6.shape)

for i in range(y_alex_vbmf_6.shape[0]):
    y_alex_vbmf_6_masked[i,:,:,:] = y_alex_vbmf_6[i,:,:,:]* mask


with h5py.File(os.path.join(config['save_path'], f'alex_vbmf_6.h5'), 'w') as f:
    f.create_dataset('y_pred', data=y_alex_vbmf_6)
    f.create_dataset('y_pred_masked', data=y_alex_vbmf_6_masked)
print(y_alex_vbmf_6.shape)

# fig, ax = plt.subplots(3,2, figsize=(10,5))
# for i in range(2):
#     im0 = ax[0,i].imshow(y_alex_vbmf_6_masked[i,10,:,:], cmap='viridis')
#     im1 = ax[1,i].imshow(y_alex_vbmf_6[i,10,:,:], cmap='viridis')
#     im2 = ax[2,i].imshow(gt[i,10,:,:], cmap="viridis")
#
#     # Add colorbars
#     plt.colorbar(im0, ax=ax[0,i], shrink=0.6)
#     plt.colorbar(im1, ax=ax[1,i], shrink=0.6)
#     plt.colorbar(im2, ax=ax[2,i], shrink=0.6)
#
#     # Optional: Add titles for clarity
#     if i < len(maps) and maps[i] != '':
#         ax[0,i].set_title(f'{maps[i]} (masked)')
#         ax[1,i].set_title(f'{maps[i]} (unmasked)')
#         ax[2,i].set_title(f'{maps[i]} (ground truth)')
#
# plt.tight_layout()
# print(y_alex_nbmf_30.shape)


#
#
# Alex vbmf 30:
alex_vbmf_30_path = f"/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/alex/vbmf_phantom-larg2_30.h5"
alex_vbmf_30_file = h5py.File(alex_vbmf_30_path, 'r')
print(alex_vbmf_30_file.keys())

y_alex_vbmf_30 = np.zeros(config['alex_image_size'])

for i,map  in enumerate(maps_alex):
    if np.any(np.isnan(alex_vbmf_30_file[map][:])):
        print(f"Warning: NaN values found in map {map}. Replacing NaNs with zeros.")
        y_alex_vbmf_30[i,:,:,:] = np.nan_to_num(alex_vbmf_30_file[map][:], nan=0.0)
    else:
        y_alex_vbmf_30[i,:,:,:] = alex_vbmf_30_file[map][:]
    print(f"for map {map}, min: {np.min(y_alex_vbmf_30[i,:,:,:])}, max: {np.max(y_alex_vbmf_30[i,:,:,:])}, shape of original: {alex_vbmf_30_file[map][:].shape}")

y_alex_vbmf_30 = np.transpose(y_alex_vbmf_30, (0,2,1,3))
y_alex_vbmf_30_masked= np.zeros(y_alex_vbmf_30.shape)
#
for i in range(y_alex_vbmf_30.shape[0]):
    y_alex_vbmf_30_masked[i,:,:,:] = y_alex_vbmf_30[i,:,:,:] * mask

with h5py.File(os.path.join(config['save_path'], f'alex_vbmf_30.h5'), 'w') as f:
    f.create_dataset('y_pred', data=y_alex_vbmf_30)
    f.create_dataset('y_pred_masked', data=y_alex_vbmf_30_masked)
print(y_alex_vbmf_30.shape)


# fig, ax = plt.subplots(3,2, figsize=(10,5))
# for i in range(2):
#     im0 = ax[0,i].imshow(y_alex_vbmf_6_masked[i,10,:,:], cmap='viridis')
#     im1 = ax[1,i].imshow(y_alex_vbmf_6[i,10,:,:], cmap='viridis')
#     im2 = ax[2,i].imshow(gt[i,10,:,:], cmap="viridis")
#
#     # Add colorbars
#     plt.colorbar(im0, ax=ax[0,i], shrink=0.6)
#     plt.colorbar(im1, ax=ax[1,i], shrink=0.6)
#     plt.colorbar(im2, ax=ax[2,i], shrink=0.6)
#
#     # Optional: Add titles for clarity
#     if i < len(maps) and maps[i] != '':
#         ax[0,i].set_title(f'{maps[i]} (masked)')
#         ax[1,i].set_title(f'{maps[i]} (unmasked)')
#         ax[2,i].set_title(f'{maps[i]} (ground truth)')
#
# plt.tight_layout()
# print(y_alex_nbmf_30.shape)



# Display figure

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from transformer_architecture_prod import *
from human.functions_prod import *
import matplotlib.colors as mcolors
from matplotlib import cm

def create_cmaps() -> list:
    # Define colormaps (using your existing ones)
    buda_map = LinearSegmentedColormap.from_list('buda', cm_data_vik)
    lipari_map = LinearSegmentedColormap.from_list('lipari', cm_data_brok)

    original_map = plt.get_cmap('viridis')
    color_mat = original_map(np.arange(original_map.N))
    color_mat[0, 0:3] = 0  # minimum value is set to black
    b_viridis = mcolors.LinearSegmentedColormap.from_list('colormap', color_mat)

    original_map = buda_map
    color_mat = original_map(np.arange(original_map.N))
    color_mat[0, 0:3] = 0  # minimum value is set to black
    b_bwr = mcolors.LinearSegmentedColormap.from_list('colormap', color_mat)

    original_map = lipari_map
    color_mat = original_map(np.arange(original_map.N))
    color_mat[0, 0:3] = 0  # minimum value is set to black
    b_rdgy = mcolors.LinearSegmentedColormap.from_list('colormap', color_mat)

    original_map = plt.get_cmap('winter')
    color_mat = original_map(np.arange(original_map.N))
    color_mat[0, 0:3] = 0  # minimum value is set to black
    winter = mcolors.LinearSegmentedColormap.from_list('colormap', color_mat)

    original_map = plt.get_cmap('hot')
    color_mat = original_map(np.arange(original_map.N))
    color_mat[0, 0:3] = 0  # minimum value is set to black
    b_hot = mcolors.LinearSegmentedColormap.from_list('colormap', color_mat)

    # Convert 'magma' string to proper colormap object
    magma_cmap = plt.get_cmap('magma')

    # Use your existing colormaps
    cbar_list = [magma_cmap, b_viridis, b_bwr, b_rdgy, b_hot, winter]
    return cbar_list







In [ ]:
fig, axs = plt.subplots(nrows=8, ncols=6, figsize=(30, 18))
fig.patch.set_facecolor('black')# Set black background
for i in range(8):
    for j in range(6):
        axs[i, j].set_facecolor('black')

cbar_list = create_cmaps()
# Define value ranges for each parameter
vmin = [0, 0, -0.6, 0.5, 2200, 400]
vmax = [1400, 120, 0.6, 1.5, 3400, 1000]

column_titles = [
    r"$K_{ssw}$ (S$^{-1}$)",
    r"$f_s$ (%)",
    r"$B_0$ (ppm)",
    r"$B_1$ (rel.)",
    r"$T_1$ (ms)",
    r"$T_2$ (ms)"
]


# Rotate TMBF data to align with other methods
tbmf_rotated = np.rot90(tbmf, k=1, axes=(2, 3))  # Rotate 90 degrees counterclockwise
tbmf_rotated_flipped = np.flip(tbmf_rotated, axis=2)  # Flip vertically

# Create a copy of the original tbmf for selective rotation
tbmf_selective = tbmf_rotated_flipped.copy()

# Keep the original (non-rotated) data for B0 and B1 (indices 2 and 3)
tbmf_selective[2] = tbmf[2]  # B0 - keep original
tbmf_selective[3] = tbmf[3]  # B1 - keep original

# Rotate TMBF data to align with other methods
tbmf_rotated1 = np.rot90(tbmf_display, k=1, axes=(2, 3))  # Rotate 90 degrees counterclockwise
tbmf_rotated_flipped1 = np.flip(tbmf_rotated1, axis=2)  # Flip vertically

# Create a copy of the original tbmf for selective rotation
tbmf_selective_display = tbmf_rotated_flipped.copy()

# Keep the original (non-rotated) data for B0 and B1 (indices 2 and 3)
tbmf_selective_display[2] = tbmf_display[2]  # B0 - keep original
tbmf_selective_display[3] = tbmf_display[3]  # B1 - keep original

with h5py.File("/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/tbmf_rotated.h5", 'w') as f:
    f.create_dataset("tbmf", data = tbmf_selective)

# Update the data_list to use selectively rotated TMBF
data_list = [gt_disp, tbmf_selective_display, y_nikita_6_display, y_nikita_30_display, y_alex_nbmf_6_masked, y_alex_nbmf_30_masked, y_alex_vbmf_6_masked, y_alex_vbmf_30_masked]

row_titles = ["GT", "TBMF", "dot-6", "dot-30", "NBMf 6", "NBMf 30", "VBMF 6", "VBMF 30"]

scale_factor_alex = [1, 110e3/3]
scale_factor_nikita = [1, 110e3/3, 1, 1, 1000, 1000]
index = 10
mask_index = 8
# Main plotting loop
for i in range(8):
    curr_data = data_list[i]

    for j in range(6):
        # Set column titles only for the first row
        if i == 0:
            axs[i, j].set_title(column_titles[j], fontsize=20, color='white', pad=20)

        # Display the image
        if i < 2:  # GT and TMBF
            if j==2 or j==3:
                img = axs[i, j].imshow(curr_data[j, index, :, :]*mask_3D[mask_index] , cmap=cbar_list[j], vmin=vmin[j], vmax=vmax[j])
            else:
                img = axs[i, j].imshow(curr_data[j, index, :, :] , cmap=cbar_list[j], vmin=vmin[j], vmax=vmax[j])

        elif i >= 4:  # Alex methods
            if j < curr_data.shape[0]:  # Only plot if the map exists
                img = axs[i, j].imshow(curr_data[j, index, :, :] * scale_factor_alex[j], cmap=cbar_list[j], vmin=vmin[j], vmax=vmax[j])
            else:
                # Display blank/black image for missing maps
                axs[i, j].imshow(np.zeros((80, 80)), cmap='gray', vmin=0, vmax=1)

        else:  # Nikita methods (rows 2 and 3: dot_6 and dot_30)
            if j == 0:
                data = (curr_data[j,index, :, :]*scale_factor_nikita[j])
                cmap = cbar_list[j]
                if isinstance(cmap, str):
                    cmap = plt.get_cmap(cmap)
                cmap.set_bad('black')
                img = axs[i, j].imshow(data, cmap=cmap, vmin=vmin[j], vmax=vmax[j])

            elif j == 1:
                data = curr_data[j, index, :, :]*scale_factor_nikita[j]
                cmap = cbar_list[j]
                if isinstance(cmap, str):
                    cmap = plt.get_cmap(cmap)
                cmap.set_bad('black')
                img = axs[i, j].imshow(data, cmap=cmap, vmin=vmin[j], vmax=vmax[j])

            elif j == 2:
                data = (curr_data[j, index, :,: ]*scale_factor_nikita[j])
                cmap = cbar_list[j]
                if isinstance(cmap, str):
                    cmap = plt.get_cmap(cmap)
                cmap.set_bad('black')
                img = axs[i, j].imshow(data*mask_3D[mask_index], cmap=cmap, vmin=vmin[j], vmax=vmax[j])

            elif j == 4:
                data = (curr_data[j, index, :, :]*scale_factor_nikita[j])
                cmap = cbar_list[j]
                if isinstance(cmap, str):
                    cmap = plt.get_cmap(cmap)
                cmap.set_bad('black')
                img = axs[i, j].imshow(data, cmap=cmap, vmin=vmin[j], vmax=vmax[j])
            elif j == 5:
                data = curr_data[j, index, :, :]*scale_factor_nikita[j]
                cmap = cbar_list[j]
                if isinstance(cmap, str):
                    cmap = plt.get_cmap(cmap)
                cmap.set_bad('black')
                img = axs[i, j].imshow(data, cmap=cmap, vmin=vmin[j], vmax=vmax[j])
            else:
                img = axs[i, j].imshow(curr_data[j, index, :,:]*scale_factor_nikita[j], cmap=cbar_list[j], vmin=vmin[j], vmax=vmax[j])



        # Handle axis appearance
        if j == 0:
            # Keep ylabel visible but remove ticks and spines
            axs[i, j].set_xticks([])
            axs[i, j].set_yticks([])
            for spine in axs[i, j].spines.values():
                spine.set_visible(False)
        else:
            axs[i, j].axis('off')

# Set row titles AFTER the main loop to avoid conflicts
# Set row titles AFTER the main loop to avoid conflicts
for i in range(8):
    axs[i, 0].set_ylabel(row_titles[i], fontsize=20, color='white', rotation=0, labelpad=50, va='center')


# Create colorbars ONCE, outside all loops
for j in range(6):
    # Get the position of the bottom subplot in each column
    bottom_ax_pos = axs[-1, j].get_position()  # Last row (index -1)

    cax = fig.add_axes([
        bottom_ax_pos.x0,  # Same left position as subplot
        bottom_ax_pos.y0 - 0.06,  # Below the subplot with spacing
        bottom_ax_pos.width+0.01,  # Same width as subplot
        0.02  # Height of colorbar
    ])

    cax.set_facecolor('black')
    sm = cm.ScalarMappable(cmap=cbar_list[j], norm=plt.Normalize(vmin=vmin[j], vmax=vmax[j]))
    sm.set_array([])
    cbar = plt.colorbar(sm, cax=cax, orientation='horizontal', ticks=np.linspace(vmin[j], vmax[j], 5))
    cbar.ax.tick_params(colors='white')
    cbar.outline.set_edgecolor('white')

#save image:
save_path = os.path.join(config['save_path'], f"phantom_method_comparison.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
plt.show()

# ICC calculation

In [ ]:
def slicewise_icc(map1, map2, icctype='ICC2'):
    """ ICC3.1 measures "consistency" while ICC2.1 measures "absolute agreement".
    """
    import pingouin as pg

    # Flatten the arrays
    map1 = map1.flatten()
    map2 = map2.flatten()

    # Create mask for valid (non-NaN) values in BOTH arrays
    valid_mask = ~np.isnan(map1) & ~np.isnan(map2)

    # Check if we have enough valid data points
    if np.sum(valid_mask) < 3:  # Need at least 3 points for ICC
        return np.nan

    # Extract only valid values
    slice1 = map1[valid_mask]
    slice2 = map2[valid_mask]

    # Create DataFrame for ICC calculation
    data = pd.DataFrame({
        'targets': np.repeat(np.arange(len(slice1)), 2),
        'raters': np.tile(['slice1', 'slice2'], len(slice1)),
        'ratings': np.concatenate([slice1, slice2])
    })

    try:
        # Compute the ICC
        icc = pg.intraclass_corr(data=data, targets='targets', raters='raters',
                                ratings='ratings', nan_policy='omit')

        # Filter for correct type and store the result
        icc_x = icc[icc['Type'] == icctype]
        if len(icc_x) > 0:
            icc_result = icc_x['ICC'].values[0]
        else:
            icc_result = np.nan
    except:
        icc_result = np.nan

    return icc_result

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

# Set up the methods and their data
methods = {
    'GT vs TBMF': (gt, tbmf_selective, 6),
    'GT vs DOT_6': (gt, y_nikita_6_masked, 6),
    'GT vs DOT_30': (gt, y_nikita_30_masked, 6),
    'GT vs NBMF_6': (gt[:2,:,:,:], y_alex_nbmf_6_masked, 2),
    'GT vs NBMF_30': (gt[:2,:,:,:], y_alex_nbmf_30_masked, 2),
    'GT vs VBMF_6': (gt[:2,:,:,:], y_alex_vbmf_6_masked, 2),
    'GT vs VBMF_30': (gt[:2,:,:,:], y_alex_vbmf_30_masked, 2)
}

# Parameter names
param_names = [
    r"$K_{ssw}$ (S$^{-1}$)",
    r"$f_s$ (%)",
    r"$B_0$ (ppm)",
    r"$B_1$ (rel.)",
    r"$T_1$ (ms)",
    r"$T_2$ (ms)"
]

# Create figure
fig, axes = plt.subplots(7, 6, figsize=(24, 28))
fig.suptitle('Statistical Analysis: ICC and Pearson Correlation vs Ground Truth',
             fontsize=16, fontweight='bold')

# Scale factors
scale_factor_alex = [1, 110e3/3]
scale_factor_nikita = [1, 110e3/3, 1, 1, 1000, 1000]

# Axis limits for each parameter [min, max]
axis_limits = {
    0: [0, 1400],      # K_ssw: 0-100
    1: [0, 120],    # f_s: 0-27.27
    2: [-0.6, 0.6],   # B_0: keep current range
    3: [0.7, 1.3],      # B_1: keep current range
    4: [2200, 3000],     # T_1: 0-3000
    5: [400, 1000]       # T_2: 0-150
}

results_summary = []

# Process each method
for method_idx, (method_name, (data1, data2, n_params)) in enumerate(methods.items()):
    print(f"\nProcessing {method_name}...")

    # Process each parameter
    for param_idx in range(6):
        ax = axes[method_idx, param_idx]

        if param_idx < n_params:
            # Skip B_1 for Nikita methods (rows 1,2)
            if (method_idx == 1 or method_idx == 2) and param_idx == 3:

                continue

            # Extract parameter data
            param1 = data1[param_idx].copy()
            param2 = data2[param_idx].copy()

            # Apply scaling to param1 (Ground Truth) - should be CONSISTENT across all methods


            # Apply scaling to param2 (comparison methods) - should be METHOD-SPECIFIC
            if method_idx == 1 or method_idx == 2:  # Nikita methods (DOT)
                if param_idx < len(scale_factor_nikita):
                    param2 = param2 * scale_factor_nikita[param_idx]
            elif method_idx >= 3:  # Alex methods (NBMF/VBMF)
                if param_idx < len(scale_factor_alex):
                    param2 = param2 * scale_factor_alex[param_idx]



            # Data validation and clipping
            if param_idx == 0:  # K_ssw should be 0-100
                param1 = np.clip(param1, 0, 1400)
                param2 = np.clip(param2, 0, 1400)
            elif param_idx == 1:  # f_s should be 0-27.27
                param1 = np.clip(param1, 0, 120)
                param2 = np.clip(param2, 0, 120)
            elif param_idx == 4:  # T_1 should be 0-3000
                param1 = np.clip(param1, 2200, 3400)
                param2 = np.clip(param2, 2200, 3400)
            elif param_idx == 5:  # T_2 should be 0-150
                param1 = np.clip(param1, 400, 1000)
                param2 = np.clip(param2, 400, 1000)

            # Right after the scaling section, add:
            print(f"\n{method_name}, Parameter {param_idx}:")
            print(f"  param1 range: [{np.nanmin(param1):.3f}, {np.nanmax(param1):.3f}]")
            print(f"  param2 range: [{np.nanmin(param2):.3f}, {np.nanmax(param2):.3f}]")
            print(f"  axis limits: {axis_limits[param_idx]}")
            print(f"  Valid points after scaling: {np.sum(~np.isnan(param1) & ~np.isnan(param2))}")

            # Calculate ICC using your original function
            try:
                icc_value = slicewise_icc(param1, param2, icctype='ICC2')
                print(f"ICC for {method_name}, {param_names[param_idx]}: {icc_value:.3f}")
            except:
                icc_value = np.nan

            # Calculate Pearson correlation
            mask = ~np.isnan(param1) & ~np.isnan(param2)
            param1_clean = param1[mask]
            param2_clean = param2[mask]

            try:
                if len(param1_clean) > 10:
                    pearson_r, pearson_p = stats.pearsonr(param1_clean, param2_clean)
                    print(f"Pearson r for {method_name}, {param_names[param_idx]}: {pearson_r:.3f}, p={pearson_p:.3e}")

                    # Create scatter plot
                    ax.scatter(param1_clean, param2_clean, alpha=0.6, s=1, color='steelblue')

                    # Set axis limits
                    ax.set_xlim(axis_limits[param_idx])
                    ax.set_ylim(axis_limits[param_idx])

                    # Add identity line within axis limits
                    lims = axis_limits[param_idx]
                    ax.plot(lims, lims, 'r--', alpha=0.8, linewidth=1)

                    # Add regression line
                    if not (np.isnan(pearson_r) or pearson_r == 0):
                        slope, intercept = np.polyfit(param1_clean, param2_clean, 1)
                        x_reg = np.array(lims)
                        y_reg = slope * x_reg + intercept
                        # Clip regression line to axis limits
                        y_reg = np.clip(y_reg, lims[0], lims[1])
                        ax.plot(x_reg, y_reg, 'orange', linewidth=1.5)

                    # Set labels and title
                    ax.set_xlabel('Ground Truth' if method_idx == len(methods)-1 else '')
                    ax.set_ylabel(method_name.split(' vs ')[1] if param_idx == 0 else '')

                    # Add statistics text
                    stats_text = f'r = {pearson_r:.3f}\n'
                    if pearson_p < 0.001:
                        stats_text += 'p < 0.001\n'
                    else:
                        stats_text += f'p = {pearson_p:.3f}\n'
                    stats_text += f'ICC = {icc_value:.3f}'

                    ax.text(0.05, 0.95, stats_text, transform=ax.transAxes,
                           fontsize=8, verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

                    # Store checkpoints
                    results_summary.append({
                        'Method': method_name,
                        'Parameter': param_names[param_idx],
                        'Pearson_r': pearson_r,
                        'Pearson_p': pearson_p,
                        'ICC': icc_value,
                        'N_points': len(param1_clean)
                    })
                else:
                    ax.text(0.5, 0.5, 'Insufficient\ndata', transform=ax.transAxes,
                           ha='center', va='center', fontsize=10)
                    ax.set_xticks([])
                    ax.set_yticks([])

            except Exception as e:
                ax.text(0.5, 0.5, 'Error in\ncalculation', transform=ax.transAxes,
                       ha='center', va='center', fontsize=10)
                ax.set_xticks([])
                ax.set_yticks([])
                print(f"Error in {method_name}, {param_names[param_idx]}: {e}")

        else:
            # Empty subplot for parameters not available in this method
            ax.axis('off')

        # Add parameter title on top row
        if method_idx == 0:
            ax.set_title(param_names[param_idx], fontweight='bold')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.95)
plt.savefig("/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/Method_ICC_single.png", dpi=300)
plt.show()

# Create summary table with error handling
results_df = pd.DataFrame(results_summary)
print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)

if not results_df.empty:
    # Pivot table for better visualization
    summary_table = results_df.pivot_table(
        index='Method',
        columns='Parameter',
        values=['Pearson_r', 'ICC'],
        aggfunc='first'
    )

    print("\nPearson Correlation Coefficients (r):")
    print("-" * 50)
    pearson_table = summary_table['Pearson_r'].round(3)
    print(pearson_table.to_string())

    print("\nIntraclass Correlation Coefficients (ICC):")
    print("-" * 50)
    if 'ICC' in summary_table.columns:
        icc_table = summary_table['ICC'].round(3)
        print(icc_table.to_string())
    else:
        icc_manual = results_df.pivot_table(
            index='Method',
            columns='Parameter',
            values='ICC',
            aggfunc='first'
        ).round(3)
        print(icc_manual.to_string())

    print("\nPearson p-values:")
    print("-" * 20)
    p_table = results_df.pivot_table(
        index='Method',
        columns='Parameter',
        values='Pearson_p',
        aggfunc='first'
    ).round(4)
    print(p_table.to_string())

    # Save checkpoints to CSV
    results_df.to_csv('/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/correlation_icc_results.csv', index=False)
    print(f"\nDetailed checkpoints saved to 'correlation_icc_results.csv'")
else:
    print("No valid checkpoints to display!")


# PSNR NRMSE and SSIM

In [ ]:
import torch
import numpy as np
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.image import PeakSignalNoiseRatio
from torch.nn.functional import mse_loss
import pandas as pd
import os

def apply_scaling(data, method, param_idx):
    """Apply scaling based on method type and parameter index"""
    if method == "gt":
        # No scaling for ground truth - keep original values
        return data
    elif method == "tbmf":
        # No scaling for TBMF - keep original values
        return data
    elif method == "nikita":
        # Scale factors for Nikita methods (DOT)
        scale_factor = [1, 110e3/3, 1, 1, 1000, 1000]
        if param_idx < len(scale_factor):
            return data * scale_factor[param_idx]
        else:
            return data
    else:  # alex methods (NBMF, VBMF)
        # Scale factors for Alex methods
        scale_factor = [1, 110e3/3]
        if param_idx < len(scale_factor):
            return data * scale_factor[param_idx]
        else:
            return data

def get_method_type(method_name):
    """Determine method type for scaling"""
    if 'TBMF' in method_name.upper():
        return 'tbmf'
    elif 'DOT' in method_name.upper():
        return 'nikita'
    else:  # NBMF, VBMF
        return 'alex'

def calc_metrics_pytorch_scaled(y_pred, y_true, method_name, param_idx, param_name=""):
    """
    Calculate metrics using PyTorch torchmetrics with proper scaling
    """
    # Apply scaling based on method type
    gt_method = 'gt'
    pred_method = get_method_type(method_name)

    # Scale both GT and predicted data
    y_true_scaled = apply_scaling(y_true.copy(), gt_method, param_idx)
    y_pred_scaled = apply_scaling(y_pred.copy(), pred_method, param_idx)

    # Get valid pixels
    valid_mask = ~np.isnan(y_pred_scaled) & ~np.isnan(y_true_scaled)

    if np.sum(valid_mask) < 100:  # Need sufficient valid pixels
        return {'psnr': np.nan, 'nrmse': np.nan, 'ssim': np.nan}

    # Extract valid values
    pred_valid = y_pred_scaled[valid_mask]
    true_valid = y_true_scaled[valid_mask]

    # Normalize both to 0-1 range based on ground truth range
    true_min, true_max = np.min(true_valid), np.max(true_valid)

    if true_max <= true_min:
        return {'psnr': np.inf, 'nrmse': 0.0, 'ssim': 1.0}  # Perfect match

    # Create full volumes with NaN replaced by mean
    pred_volume = y_pred_scaled.copy()
    true_volume = y_true_scaled.copy()
    pred_volume[~valid_mask] = np.mean(pred_valid)
    true_volume[~valid_mask] = np.mean(true_valid)

    # Normalize to 0-1 range
    true_volume_norm = (true_volume - true_min) / (true_max - true_min)
    pred_volume_norm = np.clip((pred_volume - true_min) / (true_max - true_min), 0, 1)

    # Convert to PyTorch tensors
    # For torchmetrics, we need shape (N, C, H, W) where N=batch, C=channels
    # We'll treat each depth slice as a separate sample in the batch
    depth = pred_volume_norm.shape[2]

    # Reshape: (H, W, D) -> (D, 1, H, W) for batch processing
    pred_tensor = torch.from_numpy(pred_volume_norm).float().permute(2, 0, 1).unsqueeze(1)
    true_tensor = torch.from_numpy(true_volume_norm).float().permute(2, 0, 1).unsqueeze(1)

    # Initialize metrics
    psnr_metric = PeakSignalNoiseRatio(data_range=1.0)
    ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0)

    # Calculate PSNR (can be done on entire batch)
    try:
        psnr_val = psnr_metric(pred_tensor, true_tensor).item()
    except:
        psnr_val = np.nan

    # Calculate SSIM (can be done on entire batch)
    try:
        ssim_val = ssim_metric(pred_tensor, true_tensor).item()
    except:
        ssim_val = np.nan

    # Calculate NRMSE using MSE loss
    try:
        mse_val = mse_loss(pred_tensor, true_tensor).item()
        nrmse_val = np.sqrt(mse_val)  # Already normalized since data is 0-1
    except:
        nrmse_val = np.nan

    return {
        'psnr': psnr_val,
        'nrmse': nrmse_val,
        'ssim': ssim_val
    }

def calculate_metrics_pytorch_scaled_phantom(gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
                                           y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
                                           y_alex_vbmf_6_masked, y_alex_vbmf_30_masked,
                                           save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Calculate PSNR, NRMSE, and SSIM metrics using PyTorch torchmetrics with proper scaling
    for your phantom data
    """
    # Define methods to compare using your data variables
    methods_to_compare = {
        'TBMF': {
            'data': (gt, tbmf_selective, 6),
            'n_params': 6
        },
        'DOT_6': {
            'data': (gt, y_nikita_6_masked, 6),
            'n_params': 6
        },
        'DOT_30': {
            'data': (gt, y_nikita_30_masked, 6),
            'n_params': 6
        },
        'NBMF_6': {
            'data': (gt[:2], y_alex_nbmf_6_masked, 2),
            'n_params': 2
        },
        'NBMF_30': {
            'data': (gt[:2], y_alex_nbmf_30_masked, 2),
            'n_params': 2
        },
        'VBMF_6': {
            'data': (gt[:2], y_alex_vbmf_6_masked, 2),
            'n_params': 2
        },
        'VBMF_30': {
            'data': (gt[:2], y_alex_vbmf_30_masked, 2),
            'n_params': 2
        }
    }

    param_names = ['ksw', 'fs', 'b0', 'b1', 't1', 't2']
    all_results = []  # Store all individual checkpoints for comprehensive CSV

    results = {}

    print("\nCalculating metrics using PyTorch torchmetrics with proper scaling...")
    print("=" * 75)

    for method_name, method_info in methods_to_compare.items():
        print(f"\nProcessing {method_name}...")
        results[method_name] = {}

        gt_data, pred_data, n_params = method_info['data']
        n_params = method_info['n_params']

        print(f"  GT shape: {gt_data.shape}, Pred shape: {pred_data.shape}")

        vol_results = {
            'psnr': [],
            'nrmse': [],
            'ssim': []
        }

        param_names_subset = param_names[:n_params]

        for param_idx in range(n_params):
            # Skip B1 for DOT methods (index 3)
            if method_name.startswith('DOT') and param_idx == 3:
                vol_results['psnr'].append(np.nan)
                vol_results['nrmse'].append(np.nan)
                vol_results['ssim'].append(np.nan)
                print(f"    {param_names_subset[param_idx]}: SKIPPED (B1 not available for DOT)")

                # Still add to all_results for completeness
                all_results.append({
                    'Method': method_name,
                    'Parameter': param_names_subset[param_idx],
                    'Parameter_Index': param_idx,
                    'PSNR': np.nan,
                    'NRMSE': np.nan,
                    'SSIM': np.nan,
                    'Status': 'Skipped (B1 not available)'
                })
                continue

            param_name = param_names_subset[param_idx]
            gt_param = gt_data[param_idx]  # Shape: (depth, height, width)
            pred_param = pred_data[param_idx]  # Shape: (depth, height, width)

            # Calculate metrics using PyTorch with scaling
            metrics = calc_metrics_pytorch_scaled(pred_param, gt_param, method_name, param_idx, param_name)

            vol_results['psnr'].append(metrics['psnr'])
            vol_results['nrmse'].append(metrics['nrmse'])
            vol_results['ssim'].append(metrics['ssim'])

            # Handle infinite PSNR for display
            psnr_display = "inf" if np.isinf(metrics['psnr']) else f"{metrics['psnr']:.3f}"
            print(f"    {param_name}: PSNR={psnr_display}, "
                  f"NRMSE={metrics['nrmse']:.4f}, SSIM={metrics['ssim']:.4f}")

            # Add to comprehensive checkpoints
            all_results.append({
                'Method': method_name,
                'Parameter': param_name,
                'Parameter_Index': param_idx,
                'PSNR': metrics['psnr'] if not np.isinf(metrics['psnr']) else 'inf',
                'NRMSE': metrics['nrmse'],
                'SSIM': metrics['ssim'],
                'Status': 'Calculated'
            })

        # Calculate averages (excluding inf and nan values)
        psnr_finite = [x for x in vol_results['psnr'] if np.isfinite(x)]
        nrmse_finite = [x for x in vol_results['nrmse'] if np.isfinite(x)]
        ssim_finite = [x for x in vol_results['ssim'] if np.isfinite(x)]

        avg_psnr = np.mean(psnr_finite) if psnr_finite else np.nan
        avg_nrmse = np.mean(nrmse_finite) if nrmse_finite else np.nan
        avg_ssim = np.mean(ssim_finite) if ssim_finite else np.nan

        print(f"    Average: PSNR={avg_psnr:.3f}, NRMSE={avg_nrmse:.4f}, SSIM={avg_ssim:.4f}")

        results[method_name] = vol_results

    return results, all_results, save_path

def create_summary_table_pytorch_phantom(results):
    """Create summary table for PyTorch metrics checkpoints"""
    summary_data = []

    for method_name, method_results in results.items():
        # Collect finite values only
        psnr_finite = [x for x in method_results['psnr'] if np.isfinite(x)]
        nrmse_finite = [x for x in method_results['nrmse'] if np.isfinite(x)]
        ssim_finite = [x for x in method_results['ssim'] if np.isfinite(x)]

        summary_data.append({
            'Method': method_name,
            'PSNR_mean': np.mean(psnr_finite) if psnr_finite else np.nan,
            'PSNR_std': np.std(psnr_finite) if psnr_finite else np.nan,
            'NRMSE_mean': np.mean(nrmse_finite) if nrmse_finite else np.nan,
            'NRMSE_std': np.std(nrmse_finite) if nrmse_finite else np.nan,
            'SSIM_mean': np.mean(ssim_finite) if ssim_finite else np.nan,
            'SSIM_std': np.std(ssim_finite) if ssim_finite else np.nan,
            'N_valid': len(psnr_finite)
        })

    return pd.DataFrame(summary_data)

# Main execution function
def run_phantom_metrics_analysis():
    """
    Main function to run the complete metrics analysis on your phantom data
    """
    # Make sure your data variables are available in the global scope
    # You should have: gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
    # y_alex_nbmf_6_masked, y_alex_nbmf_30_masked, y_alex_vbmf_6_masked, y_alex_vbmf_30_masked

    # Run the analysis with proper scaling
    results_scaled, all_results_data, save_path = calculate_metrics_pytorch_scaled_phantom(
        gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
        y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
        y_alex_vbmf_6_masked, y_alex_vbmf_30_masked
    )

    # Create summary
    summary_df_scaled = create_summary_table_pytorch_phantom(results_scaled)

    print("\n" + "=" * 85)
    print("PHANTOM SCALED METRICS SUMMARY (PyTorch torchmetrics)")
    print("=" * 85)

    # Format and display checkpoints table
    formatted_results_scaled = []
    for _, row in summary_df_scaled.iterrows():
        formatted_results_scaled.append({
            'Method': row['Method'],
            'PSNR': f"{row['PSNR_mean']:.3f} ± {row['PSNR_std']:.3f}",
            'NRMSE': f"{row['NRMSE_mean']:.4f} ± {row['NRMSE_std']:.4f}",
            'SSIM': f"{row['SSIM_mean']:.4f} ± {row['SSIM_std']:.4f}",
            'N': int(row['N_valid'])
        })

    formatted_df_scaled = pd.DataFrame(formatted_results_scaled)
    print(formatted_df_scaled.to_string(index=False))

    # Save comprehensive checkpoints to single CSV
    all_results_df = pd.DataFrame(all_results_data)

    # Save to CSV
    csv_filename = os.path.join(save_path, 'phantom_comprehensive_metrics.csv')
    all_results_df.to_csv(csv_filename, index=False)

    # Also save summary
    summary_csv_filename = os.path.join(save_path, 'phantom_summary_metrics.csv')
    summary_df_scaled.to_csv(summary_csv_filename, index=False)

    print(f"\nResults saved:")
    print(f"- Comprehensive data: '{csv_filename}'")
    print(f"- Summary data: '{summary_csv_filename}'")

    # Display first few rows of comprehensive data
    print(f"\nFirst 10 rows of comprehensive data:")
    print(all_results_df.head(10).to_string(index=False))

    print(f"\nTotal number of metric calculations: {len(all_results_df)}")
    print(f"Successful calculations: {len(all_results_df[all_results_df['Status'] == 'Calculated'])}")
    print(f"Skipped calculations: {len(all_results_df[all_results_df['Status'] != 'Calculated'])}")

    return results_scaled, all_results_df, summary_df_scaled

# To run the analysis, simply call:
results_scaled, all_results_df, summary_df_scaled = run_phantom_metrics_analysis()

# APE and boxplots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch

def calculate_ape(y_pred, y_true, method_name, param_idx):
    """
    Calculate Absolute Percentage Error (APE) for all valid pixels
    APE = |y_true - y_pred| / |y_true| * 100
    """
    # Apply scaling based on method type
    gt_method = 'gt'
    pred_method = get_method_type(method_name)

    # Scale both GT and predicted data
    y_true_scaled = apply_scaling(y_true.copy(), gt_method, param_idx)
    y_pred_scaled = apply_scaling(y_pred.copy(), pred_method, param_idx)

    # Get valid pixels (non-NaN and non-zero denominators)
    valid_mask = (~np.isnan(y_pred_scaled) &
                  ~np.isnan(y_true_scaled) &
                  (np.abs(y_true_scaled) > 1e-10))  # Avoid division by very small numbers

    if np.sum(valid_mask) < 10:  # Need sufficient valid pixels
        return np.array([])

    # Extract valid values
    pred_valid = y_pred_scaled[valid_mask]
    true_valid = y_true_scaled[valid_mask]

    # Calculate APE
    ape = np.abs(true_valid - pred_valid) / np.abs(true_valid) * 100

    # Remove outliers (APE > 500% might be unrealistic)
    ape_filtered = ape[ape <= 500]

    return ape_filtered

def calculate_all_ape_phantom(gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
                             y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
                             y_alex_vbmf_6_masked, y_alex_vbmf_30_masked):
    """
    Calculate APE for all methods and parameters
    """
    # Define methods to compare
    methods_to_compare = {
        'TBMF': {
            'data': (gt, tbmf_selective, 6),
            'n_params': 6
        },
        'DOT_6': {
            'data': (gt, y_nikita_6_masked, 6),
            'n_params': 6
        },
        'DOT_30': {
            'data': (gt, y_nikita_30_masked, 6),
            'n_params': 6
        },
        'NBMF_6': {
            'data': (gt[:2], y_alex_nbmf_6_masked, 2),
            'n_params': 2
        },
        'NBMF_30': {
            'data': (gt[:2], y_alex_nbmf_30_masked, 2),
            'n_params': 2
        },
        'VBMF_6': {
            'data': (gt[:2], y_alex_vbmf_6_masked, 2),
            'n_params': 2
        },
        'VBMF_30': {
            'data': (gt[:2], y_alex_vbmf_30_masked, 2),
            'n_params': 2
        }
    }

    param_names = ['K_ssw', 'f_s', 'B_0', 'B_1', 'T_1', 'T_2']
    param_units = ['(S⁻¹)', '(%)', '(ppm)', '(rel.)', '(ms)', '(ms)']

    ape_data = []
    ape_results = {}

    print("Calculating APE for all methods and parameters...")
    print("=" * 60)

    for method_name, method_info in methods_to_compare.items():
        print(f"\nProcessing {method_name}...")

        gt_data, pred_data, n_params = method_info['data']
        n_params = method_info['n_params']

        ape_results[method_name] = {}

        for param_idx in range(n_params):
            # Skip B1 for DOT methods (index 3)
            if method_name.startswith('DOT') and param_idx == 3:
                print(f"  {param_names[param_idx]}: SKIPPED (B1 not available for DOT)")
                ape_results[method_name][param_idx] = np.array([])
                continue

            param_name = param_names[param_idx]
            gt_param = gt_data[param_idx]
            pred_param = pred_data[param_idx]

            # Calculate APE
            ape_values = calculate_ape(pred_param, gt_param, method_name, param_idx)
            ape_results[method_name][param_idx] = ape_values

            if len(ape_values) > 0:
                median_ape = np.median(ape_values)
                q25_ape = np.percentile(ape_values, 25)
                q75_ape = np.percentile(ape_values, 75)
                mean_ape = np.mean(ape_values)

                print(f"  {param_name}: {len(ape_values)} pixels, "
                      f"Median APE: {median_ape:.2f}%, "
                      f"Mean APE: {mean_ape:.2f}%, "
                      f"IQR: [{q25_ape:.2f}%, {q75_ape:.2f}%]")

                # Store data for plotting
                for ape_val in ape_values:
                    ape_data.append({
                        'Method': method_name,
                        'Parameter': param_name,
                        'Parameter_Index': param_idx,
                        'APE': ape_val
                    })
            else:
                print(f"  {param_name}: No valid data")

    return ape_results, pd.DataFrame(ape_data), param_names, param_units

def create_ape_boxplots(ape_results, ape_df, param_names, param_units,
                       save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Create boxplots for APE values for each parameter across all methods
    """
    # Define colors for each method
    method_colors = {
        'TBMF': '#1f77b4',      # Blue
        'DOT_6': '#ff7f0e',     # Orange
        'DOT_30': '#d62728',    # Red
        'NBMF_6': '#2ca02c',    # Green
        'NBMF_30': '#9467bd',   # Purple
        'VBMF_6': '#8c564b',    # Brown
        'VBMF_30': '#e377c2'    # Pink
    }

    # Create figure with subplots for each parameter
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Absolute Percentage Error (APE) by Parameter and Method', fontsize=16, fontweight='bold')

    axes = axes.flatten()

    for param_idx in range(6):

        ax = axes[param_idx]
        if param_idx==3:
            ax.axis("off")
            continue

        # Filter data for current parameter
        param_data = ape_df[ape_df['Parameter_Index'] == param_idx]

        if param_data.empty:
            ax.text(0.5, 0.5, f'No data for\n{param_names[param_idx]}',
                   transform=ax.transAxes, ha='center', va='center', fontsize=12)
            ax.set_title(f'{param_names[param_idx]} {param_units[param_idx]}', fontweight='bold')
            continue

        # Create boxplot
        methods_present = param_data['Method'].unique()
        colors_for_plot = [method_colors[method] for method in methods_present]

        bp = ax.boxplot([param_data[param_data['Method'] == method]['APE'].values
                        for method in methods_present],
                       tick_labels=methods_present,
                       patch_artist=True,
                       showfliers=False,  # Hide outliers for cleaner plot
                       medianprops={'color': 'black', 'linewidth': 2})

        # Color the boxes
        for patch, color in zip(bp['boxes'], colors_for_plot):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)

        # Customize the plot
        ax.set_title(f'{param_names[param_idx]} {param_units[param_idx]}', fontweight='bold')
        ax.set_ylabel('APE (%)')
        ax.grid(True, alpha=0.3)

        # Rotate x-labels if needed
        ax.tick_params(axis='x', rotation=45)

        # Set reasonable y-axis limits (up to 95th percentile to avoid extreme outliers)
        if len(param_data) > 0:
            y_max = np.percentile(param_data['APE'], 95)
            ax.set_ylim(0, y_max * 1.2)  # Cap at 100% for readability

    plt.tight_layout()

    # Save the plot
    plot_filename = f"{save_path}/ape_boxplots_by_parameter_with_IQR.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"APE boxplots saved to: {plot_filename}")

def create_ape_summary_table(ape_df):
    """
    Create a summary table with APE statistics
    """
    summary_stats = []

    for param_name in ape_df['Parameter'].unique():
        param_data = ape_df[ape_df['Parameter'] == param_name]

        for method in param_data['Method'].unique():
            method_data = param_data[param_data['Method'] == method]['APE']

            if len(method_data) > 0:
                summary_stats.append({
                    'Parameter': param_name,
                    'Method': method,
                    'Count': len(method_data),
                    'Mean_APE': np.mean(method_data),
                    'Median_APE': np.median(method_data),
                    'Q25_APE': np.percentile(method_data, 25),
                    'Q75_APE': np.percentile(method_data, 75),
                    'Min_APE': np.min(method_data),
                    'Max_APE': np.max(method_data),
                    'Std_APE': np.std(method_data)
                })

    return pd.DataFrame(summary_stats)

def create_method_comparison_boxplot(ape_df, save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Create a single boxplot comparing all methods across all parameters
    """
    plt.figure(figsize=(12, 8))

    # Define colors for each method
    method_colors = {
        'TBMF': '#1f77b4',      # Blue
        'DOT_6': '#ff7f0e',     # Orange
        'DOT_30': '#d62728',    # Red
        'NBMF_6': '#2ca02c',    # Green
        'NBMF_30': '#9467bd',   # Purple
        'VBMF_6': '#8c564b',    # Brown
        'VBMF_30': '#e377c2'    # Pink
    }

    methods_present = ape_df['Method'].unique()
    colors_for_plot = [method_colors[method] for method in methods_present]

    bp = plt.boxplot([ape_df[ape_df['Method'] == method]['APE'].values
                     for method in methods_present],
                    tick_labels=methods_present,
                    patch_artist=True,
                    showfliers=False,
                    medianprops={'color': 'black', 'linewidth': 2})

    # Color the boxes
    for patch, color in zip(bp['boxes'], colors_for_plot):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    plt.title('Overall APE Comparison Across All Methods and Parameters', fontsize=14, fontweight='bold')
    plt.ylabel('APE (%)')
    plt.xlabel('Method')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)

    # Set reasonable y-axis limits
    y_max = np.percentile(ape_df['APE'], 95)
    plt.ylim(0, y_max * 1.5)

    plt.tight_layout()

    # Save the plot
    plot_filename = f"{save_path}/ape_boxplots_overall_comparison_with_IQR.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Overall APE comparison saved to: {plot_filename}")

def run_ape_analysis():
    """
    Main function to run complete APE analysis
    """
    # Calculate APE for all methods and parameters
    ape_results, ape_df, param_names, param_units = calculate_all_ape_phantom(
        gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
        y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
        y_alex_vbmf_6_masked, y_alex_vbmf_30_masked
    )

    # Create boxplots by parameter
    create_ape_boxplots(ape_results, ape_df, param_names, param_units)

    # Create overall method comparison
    create_method_comparison_boxplot(ape_df)

    # Create summary table
    summary_table = create_ape_summary_table(ape_df)

    # Display summary statistics
    print("\n" + "=" * 80)
    print("APE SUMMARY STATISTICS")
    print("=" * 80)

    # Show summary for each parameter
    for param in param_names[:6]:
        param_summary = summary_table[summary_table['Parameter'] == param]
        if not param_summary.empty:
            print(f"\n{param}:")
            print("-" * 40)
            for _, row in param_summary.iterrows():
                print(f"  {row['Method']:10}: "
                      f"Median={row['Median_APE']:6.2f}%, "
                      f"Mean={row['Mean_APE']:6.2f}%, "
                      f"IQR=[{row['Q25_APE']:5.2f}%, {row['Q75_APE']:5.2f}%], "
                      f"N={row['Count']:6d}")

    # Save detailed checkpoints
    save_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"

    # Save comprehensive APE data
    ape_csv_filename = f"{save_path}/ape_comprehensive_data_with_IQR.csv"
    ape_df.to_csv(ape_csv_filename, index=False)

    # Save summary statistics
    summary_csv_filename = f"{save_path}/ape_summary_statistics_with_IQR.csv"
    summary_table.to_csv(summary_csv_filename, index=False)

    print(f"\nResults saved:")
    print(f"- Comprehensive APE data: '{ape_csv_filename}'")
    print(f"- Summary statistics: '{summary_csv_filename}'")

    return ape_results, ape_df, summary_table

# To run the complete APE analysis:
ape_results, ape_df, summary_table = run_ape_analysis()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch

def calculate_ape_slice_based(y_pred, y_true, method_name, param_idx):
    """
    Calculate Absolute Percentage Error (APE) based on slice averages
    APE = |y_true_avg - y_pred_avg| / |y_true_avg| * 100
    Returns one APE value per slice (12 values total)
    """
    # Apply scaling based on method type
    gt_method = 'gt'
    pred_method = get_method_type(method_name)

    # Scale both GT and predicted data
    y_true_scaled = apply_scaling(y_true.copy(), gt_method, param_idx)
    y_pred_scaled = apply_scaling(y_pred.copy(), pred_method, param_idx)

    if y_true_scaled.ndim == 3:
        # Find slice dimension with 12 slices
        slice_dim = None
        for dim in [0, 1, 2]:
            if y_true_scaled.shape[dim] == 12:
                slice_dim = dim
                break

        if slice_dim is None:
            slice_dim = -1
        n_slices = y_true_scaled.shape[slice_dim]

    elif y_true_scaled.ndim == 2:
        n_slices = 1
        slice_dim = None
    else:
        return np.array([])

    slice_ape_values = []

    for slice_idx in range(n_slices):
        # Extract current slice based on slice dimension
        if y_true_scaled.ndim == 3:
            if slice_dim == 0:
                true_slice = y_true_scaled[slice_idx, :, :]
                pred_slice = y_pred_scaled[slice_idx, :, :]
            elif slice_dim == 1:
                true_slice = y_true_scaled[:, slice_idx, :]
                pred_slice = y_pred_scaled[:, slice_idx, :]
            else:  # slice_dim == 2 or -1
                true_slice = y_true_scaled[:, :, slice_idx]
                pred_slice = y_pred_scaled[:, :, slice_idx]
        else:
            true_slice = y_true_scaled
            pred_slice = y_pred_scaled

        # Get valid pixels in this slice
        valid_mask = (~np.isnan(pred_slice) &
                      ~np.isnan(true_slice) &
                      (np.abs(true_slice) > 1e-10))

        n_valid = np.sum(valid_mask)

        if n_valid < 5:  # Need at least 5 valid pixels per slice
            continue

        # Calculate slice averages for valid pixels only
        true_slice_avg = np.mean(true_slice[valid_mask])
        pred_slice_avg = np.mean(pred_slice[valid_mask])

        # Calculate APE for this slice
        if np.abs(true_slice_avg) > 1e-10:
            slice_ape = np.abs(true_slice_avg - pred_slice_avg) / np.abs(true_slice_avg) * 100

            # Remove unrealistic values
            if slice_ape <= 500:
                slice_ape_values.append(slice_ape)

    return np.array(slice_ape_values)

def calculate_all_ape_phantom_slice_based(gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
                                         y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
                                         y_alex_vbmf_6_masked, y_alex_vbmf_30_masked):
    """
    Calculate slice-based APE for all methods and parameters
    """
    # Define methods to compare
    methods_to_compare = {
        'TBMF': {
            'data': (gt, tbmf_selective, 6),
            'n_params': 6
        },
        'DOT_6': {
            'data': (gt, y_nikita_6_masked, 6),
            'n_params': 6
        },
        'DOT_30': {
            'data': (gt, y_nikita_30_masked, 6),
            'n_params': 6
        },
        'NBMF_6': {
            'data': (gt[:2], y_alex_nbmf_6_masked, 2),
            'n_params': 2
        },
        'NBMF_30': {
            'data': (gt[:2], y_alex_nbmf_30_masked, 2),
            'n_params': 2
        },
        'VBMF_6': {
            'data': (gt[:2], y_alex_vbmf_6_masked, 2),
            'n_params': 2
        },
        'VBMF_30': {
            'data': (gt[:2], y_alex_vbmf_30_masked, 2),
            'n_params': 2
        }
    }

    param_names = ['K_sw', 'f_s', 'B_0', 'B_1', 'T_1', 'T_2']
    param_units = ['(S⁻¹)', '(%)', '(ppm)', '(rel.)', '(ms)', '(ms)']

    ape_data = []
    ape_results = {}

    print("Calculating slice-based APE for all methods and parameters...")

    for method_name, method_info in methods_to_compare.items():
        print(f"Processing {method_name}...")

        gt_data, pred_data, n_params = method_info['data']
        n_params = method_info['n_params']
        ape_results[method_name] = {}

        for param_idx in range(n_params):
            # Skip B1 for DOT methods (index 3)
            if method_name.startswith('DOT') and param_idx == 3:
                ape_results[method_name][param_idx] = np.array([])
                continue

            param_name = param_names[param_idx]

            try:
                gt_param = gt_data[param_idx]
                pred_param = pred_data[param_idx]

                # Calculate slice-based APE
                ape_values = calculate_ape_slice_based(pred_param, gt_param, method_name, param_idx)
                ape_results[method_name][param_idx] = ape_values

                if len(ape_values) > 0:
                    median_ape = np.median(ape_values)
                    q25_ape = np.percentile(ape_values, 25)
                    q75_ape = np.percentile(ape_values, 75)
                    mean_ape = np.mean(ape_values)

                    print(f"  {param_name}: {len(ape_values)} slices, "
                          f"Median APE: {median_ape:.2f}%, "
                          f"Mean APE: {mean_ape:.2f}%, "
                          f"IQR: [{q25_ape:.2f}%, {q75_ape:.2f}%]")

                    # Store data for plotting
                    for slice_idx, ape_val in enumerate(ape_values):
                        ape_data.append({
                            'Method': method_name,
                            'Parameter': param_name,
                            'Parameter_Index': param_idx,
                            'Slice': slice_idx,
                            'APE': ape_val
                        })

            except Exception as e:
                print(f"    ERROR processing parameter {param_idx}: {str(e)}")
                ape_results[method_name][param_idx] = np.array([])

    return ape_results, pd.DataFrame(ape_data), param_names, param_units

def create_ape_boxplots_slice_based_with_points(ape_results, ape_df, param_names, param_units,
                                               save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Create boxplots for slice-based APE values with individual data points overlaid
    """
    # Define colors for each method
    method_colors = {
        'TBMF': '#1f77b4',      # Blue
        'DOT_6': '#ff7f0e',     # Orange
        'DOT_30': '#d62728',    # Red
        'NBMF_6': '#2ca02c',    # Green
        'NBMF_30': '#9467bd',   # Purple
        'VBMF_6': '#8c564b',    # Brown
        'VBMF_30': '#e377c2'    # Pink
    }

    # Create figure with black background
    fig, axes = plt.subplots(2, 3, figsize=(18, 12), facecolor='black')
    fig.suptitle('Slice-based Absolute Percentage Error (APE) by Parameter and Method',
                 fontsize=16, fontweight='bold', color='white')

    axes = axes.flatten()

    for param_idx in range(6):
        ax = axes[param_idx]
        ax.set_facecolor('black')  # Set subplot background to black

        if param_idx == 3:
            ax.axis("off")
            continue

        # Filter data for current parameter
        param_data = ape_df[ape_df['Parameter_Index'] == param_idx]

        if param_data.empty:
            ax.text(0.5, 0.5, f'No data for\n{param_names[param_idx]}',
                   transform=ax.transAxes, ha='center', va='center',
                   fontsize=12, color='white')
            ax.set_title(f'{param_names[param_idx]} {param_units[param_idx]}',
                        fontweight='bold', color='white')
            continue

        # Create boxplot without outliers
        methods_present = param_data['Method'].unique()
        colors_for_plot = [method_colors[method] for method in methods_present]

        bp = ax.boxplot([param_data[param_data['Method'] == method]['APE'].values
                        for method in methods_present],
                       tick_labels=methods_present,
                       patch_artist=True,
                       showfliers=False,
                       medianprops={'color': 'white', 'linewidth': 2},
                       boxprops={'color': 'white'},
                       whiskerprops={'color': 'white'},
                       capprops={'color': 'white'})

        # Color the boxes
        for patch, color in zip(bp['boxes'], colors_for_plot):
            patch.set_facecolor('none')  # Remove fill color
            patch.set_edgecolor("black")   # Keep colored outline
            patch.set_linewidth(2)       # Make outline more visible

        # Add individual data points
        for i, method in enumerate(methods_present):
            method_data = param_data[param_data['Method'] == method]['APE'].values
            x_coords = np.random.normal(i + 1, 0.04, len(method_data))
            ax.scatter(x_coords, method_data,
                      color=method_colors[method],
                      alpha=0.7,
                      s=30,
                      edgecolors='white',
                      linewidth=0.5,
                      zorder=3)

        # Customize the plot
        ax.set_title(f'{param_names[param_idx]} {param_units[param_idx]}',
                    fontweight='bold', color='white')
        ax.set_ylabel('APE (%) - Slice Average', color='white')
        ax.grid(True, alpha=0.3, color='gray')
        ax.tick_params(axis='x', rotation=45, colors='white')
        ax.tick_params(axis='y', colors='white')

        # Set reasonable y-axis limits
        if len(param_data) > 0:
            y_max = np.percentile(param_data['APE'], 95)
            ax.set_ylim(0, max(y_max * 1.2, np.max(param_data['APE']) * 1.1))

    plt.tight_layout()

    # Save the plot
    plot_filename = f"{save_path}/ape_boxplots_slice_based_with_points_black.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight', facecolor='black')
    plt.show()

    print(f"Slice-based APE boxplots with points saved to: {plot_filename}")

def create_method_comparison_boxplot_slice_based_with_points(ape_df, save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Create a single boxplot comparing all methods with points colored by parameter, ordered by performance
    """
    fig = plt.figure(figsize=(14, 10), facecolor='white')
    ax = fig.add_subplot(111)
    ax.set_facecolor('white')

    # Define colors for each method
    method_colors = {
        'TBMF': '#1f77b4',      # Blue
        'DOT_6': '#ff7f0e',     # Orange
        'DOT_30': '#d62728',    # Red
        'NBMF_6': '#2ca02c',    # Green
        'NBMF_30': '#9467bd',   # Purple
        'VBMF_6': '#8c564b',    # Brown
        'VBMF_30': '#e377c2'    # Pink
    }

    # Define colors for each parameter
    param_colors = {
        'K_sw': '#1f77b4',     # Blue
        'f_s': '#ff7f0e',       # Orange
        'B_0': '#2ca02c',       # Green
        'B_1': '#d62728',       # Red
        'T_1': '#9467bd',       # Purple
        'T_2': '#8c564b'        # Brown
    }

    # Calculate median APE for each method and sort from best to worst
    method_medians = []
    for method in ape_df['Method'].unique():
        method_data = ape_df[ape_df['Method'] == method]['APE']
        median_ape = np.median(method_data)
        method_medians.append((method, median_ape))

    # Sort by median APE (ascending - best to worst)
    method_medians.sort(key=lambda x: x[1])
    methods_present = [method for method, _ in method_medians]

    colors_for_plot = [method_colors[method] for method in methods_present]

    # Create boxplot with boxes visible
    bp = ax.boxplot([ape_df[ape_df['Method'] == method]['APE'].values
                     for method in methods_present],
                    tick_labels=methods_present,
                    patch_artist=True,
                    showfliers=False,
                    medianprops={'color': 'black', 'linewidth': 2},
                    boxprops={'color': 'black'},
                    whiskerprops={'color': 'black'},
                    capprops={'color': 'black'})

    # Make boxes transparent (no fill)
    for patch, color in zip(bp['boxes'], colors_for_plot):
        patch.set_facecolor('none')  # Remove fill color
        patch.set_edgecolor("black")   # Keep colored outline
        patch.set_linewidth(2)       # Make outline more visible

    # Add individual data points colored by parameter
    for i, method in enumerate(methods_present):
        method_data = ape_df[ape_df['Method'] == method]

        for param in method_data['Parameter'].unique():
            param_data = method_data[method_data['Parameter'] == param]
            x_coords = np.random.normal(i + 1, 0.08, len(param_data))
            ax.scatter(x_coords, param_data['APE'].values,
                       color=param_colors[param],
                       alpha=0.8,
                       s=50,
                       edgecolors='black',
                       linewidth=0.5,
                       label=param if i == 0 else "",
                       zorder=3)

    ax.set_title('Overall Slice-based APE Comparison Across All Methods and Parameters\n(Ordered by performance, points colored by parameter)',
                fontsize=14, fontweight='bold', color='black')
    ax.set_ylabel('APE (%) - Slice Average', color='black')
    ax.set_xlabel('Method', color='black')
    ax.grid(True, alpha=0.3, color='gray')
    ax.tick_params(axis='x', rotation=45, colors='black')
    ax.tick_params(axis='y', colors='black')

    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)

    # Set reasonable y-axis limits
    y_max = np.percentile(ape_df['APE'], 95)
    ax.set_ylim(0, max(y_max * 1.5, np.max(ape_df['APE']) * 1.1))

    # Add legend for parameters inside the plot with no background
    legend_elements = [plt.scatter([], [], color=param_colors[param], s=60, alpha=0.8,
                                 edgecolors='black', linewidth=0.5, label=param)
                      for param in sorted(param_colors.keys())]
    legend = ax.legend(handles=legend_elements, title='Parameters',
                      loc='upper right', frameon=False)  # frameon=False removes white box
    legend.get_title().set_color('black')
    for text in legend.get_texts():
        text.set_color('black')

    plt.tight_layout()

    # Save the plot
    plot_filename = f"{save_path}/ape_boxplots_slice_based_overall_with_points_ordered_black.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

    print(f"Overall slice-based APE comparison with points (ordered) saved to: {plot_filename}")

def create_ape_summary_table_slice_based(ape_df):
    """
    Create a summary table with slice-based APE statistics
    """
    summary_stats = []

    for param_name in ape_df['Parameter'].unique():
        param_data = ape_df[ape_df['Parameter'] == param_name]

        for method in param_data['Method'].unique():
            method_data = param_data[param_data['Method'] == method]['APE']

            if len(method_data) > 0:
                summary_stats.append({
                    'Parameter': param_name,
                    'Method': method,
                    'N_Slices': len(method_data),
                    'Mean_APE': np.mean(method_data),
                    'Median_APE': np.median(method_data),
                    'Q25_APE': np.percentile(method_data, 25),
                    'Q75_APE': np.percentile(method_data, 75),
                    'Min_APE': np.min(method_data),
                    'Max_APE': np.max(method_data),
                    'Std_APE': np.std(method_data)
                })

    return pd.DataFrame(summary_stats)

def run_ape_analysis_slice_based_with_points():
    """
    Main function to run complete slice-based APE analysis with individual points
    """
    # Calculate slice-based APE for all methods and parameters
    ape_results, ape_df, param_names, param_units = calculate_all_ape_phantom_slice_based(
        gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
        y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
        y_alex_vbmf_6_masked, y_alex_vbmf_30_masked
    )

    # Create boxplots by parameter with individual points
    create_ape_boxplots_slice_based_with_points(ape_results, ape_df, param_names, param_units)

    # Create overall method comparison with points colored by parameter
    create_method_comparison_boxplot_slice_based_with_points(ape_df)

    # Create summary table
    summary_table = create_ape_summary_table_slice_based(ape_df)

    # Display summary statistics
    print("\n" + "=" * 80)
    print("SLICE-BASED APE SUMMARY STATISTICS")
    print("=" * 80)

    # Show summary for each parameter
    for param in param_names[:6]:
        param_summary = summary_table[summary_table['Parameter'] == param]
        if not param_summary.empty:
            print(f"\n{param}:")
            print("-" * 40)
            for _, row in param_summary.iterrows():
                print(f"  {row['Method']:10}: "
                      f"Median={row['Median_APE']:6.2f}%, "
                      f"Mean={row['Mean_APE']:6.2f}%, "
                      f"IQR=[{row['Q25_APE']:5.2f}%, {row['Q75_APE']:5.2f}%], "
                      f"N_slices={row['N_Slices']:2d}")

    # Save detailed checkpoints
    save_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"

    # Save comprehensive APE data
    ape_csv_filename = f"{save_path}/ape_slice_based_comprehensive_data_with_points.csv"
    ape_df.to_csv(ape_csv_filename, index=False)

    # Save summary statistics
    summary_csv_filename = f"{save_path}/ape_slice_based_summary_statistics_with_points.csv"
    summary_table.to_csv(summary_csv_filename, index=False)

    print(f"\nResults saved:")
    print(f"- Comprehensive slice-based APE data: '{ape_csv_filename}'")
    print(f"- Summary statistics: '{summary_csv_filename}'")

    return ape_results, ape_df, summary_table

# To run the complete slice-based APE analysis with individual points:
ape_results, ape_df, summary_table = run_ape_analysis_slice_based_with_points()

# one way anova

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch
import scipy.stats as stats
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def calculate_ape_slice_based(y_pred, y_true, method_name, param_idx):
    """
    Calculate Absolute Percentage Error (APE) based on slice averages
    APE = |y_true_avg - y_pred_avg| / |y_true_avg| * 100
    Returns one APE value per slice (12 values total)
    """
    # Apply scaling based on method type
    gt_method = 'gt'
    pred_method = get_method_type(method_name)

    # Scale both GT and predicted data
    y_true_scaled = apply_scaling(y_true.copy(), gt_method, param_idx)
    y_pred_scaled = apply_scaling(y_pred.copy(), pred_method, param_idx)

    if y_true_scaled.ndim == 3:
        # Find slice dimension with 12 slices
        slice_dim = None
        for dim in [0, 1, 2]:
            if y_true_scaled.shape[dim] == 12:
                slice_dim = dim
                break

        if slice_dim is None:
            slice_dim = -1
        n_slices = y_true_scaled.shape[slice_dim]

    elif y_true_scaled.ndim == 2:
        n_slices = 1
        slice_dim = None
    else:
        return np.array([])

    slice_ape_values = []

    for slice_idx in range(n_slices):
        # Extract current slice based on slice dimension
        if y_true_scaled.ndim == 3:
            if slice_dim == 0:
                true_slice = y_true_scaled[slice_idx, :, :]
                pred_slice = y_pred_scaled[slice_idx, :, :]
            elif slice_dim == 1:
                true_slice = y_true_scaled[:, slice_idx, :]
                pred_slice = y_pred_scaled[:, slice_idx, :]
            else:  # slice_dim == 2 or -1
                true_slice = y_true_scaled[:, :, slice_idx]
                pred_slice = y_pred_scaled[:, :, slice_idx]
        else:
            true_slice = y_true_scaled
            pred_slice = y_pred_scaled

        # Get valid pixels in this slice
        valid_mask = (~np.isnan(pred_slice) &
                      ~np.isnan(true_slice) &
                      (np.abs(true_slice) > 1e-10))

        n_valid = np.sum(valid_mask)

        if n_valid < 5:  # Need at least 5 valid pixels per slice
            continue

        # Calculate slice averages for valid pixels only
        true_slice_avg = np.mean(true_slice[valid_mask])
        pred_slice_avg = np.mean(pred_slice[valid_mask])

        # Calculate APE for this slice
        if np.abs(true_slice_avg) > 1e-10:
            slice_ape = np.abs(true_slice_avg - pred_slice_avg) / np.abs(true_slice_avg) * 100

            # Remove unrealistic values
            if slice_ape <= 500:
                slice_ape_values.append(slice_ape)

    return np.array(slice_ape_values)

def calculate_all_ape_phantom_slice_based(gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
                                         y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
                                         y_alex_vbmf_6_masked, y_alex_vbmf_30_masked):
    """
    Calculate slice-based APE for all methods and parameters
    """
    # Define methods to compare
    methods_to_compare = {
        'TBMF': {
            'data': (gt, tbmf_selective, 6),
            'n_params': 6
        },
        'DOT_6': {
            'data': (gt, y_nikita_6_masked, 6),
            'n_params': 6
        },
        'DOT_30': {
            'data': (gt, y_nikita_30_masked, 6),
            'n_params': 6
        },
        'NBMF_6': {
            'data': (gt[:2], y_alex_nbmf_6_masked, 2),
            'n_params': 2
        },
        'NBMF_30': {
            'data': (gt[:2], y_alex_nbmf_30_masked, 2),
            'n_params': 2
        },
        'VBMF_6': {
            'data': (gt[:2], y_alex_vbmf_6_masked, 2),
            'n_params': 2
        },
        'VBMF_30': {
            'data': (gt[:2], y_alex_vbmf_30_masked, 2),
            'n_params': 2
        }
    }

    param_names = ['K_sw', 'f_s', 'B_0', 'B_1', 'T_1', 'T_2']
    param_units = ['(S⁻¹)', '(%)', '(ppm)', '(rel.)', '(ms)', '(ms)']

    ape_data = []
    ape_results = {}

    print("Calculating slice-based APE for all methods and parameters...")

    for method_name, method_info in methods_to_compare.items():
        print(f"Processing {method_name}...")

        gt_data, pred_data, n_params = method_info['data']
        n_params = method_info['n_params']
        ape_results[method_name] = {}

        for param_idx in range(n_params):
            # Skip B1 for DOT methods (index 3)
            if method_name.startswith('DOT') and param_idx == 3:
                ape_results[method_name][param_idx] = np.array([])
                continue

            param_name = param_names[param_idx]

            try:
                gt_param = gt_data[param_idx]
                pred_param = pred_data[param_idx]

                # Calculate slice-based APE
                ape_values = calculate_ape_slice_based(pred_param, gt_param, method_name, param_idx)
                ape_results[method_name][param_idx] = ape_values

                if len(ape_values) > 0:
                    median_ape = np.median(ape_values)
                    q25_ape = np.percentile(ape_values, 25)
                    q75_ape = np.percentile(ape_values, 75)
                    mean_ape = np.mean(ape_values)

                    print(f"  {param_name}: {len(ape_values)} slices, "
                          f"Median APE: {median_ape:.2f}%, "
                          f"Mean APE: {mean_ape:.2f}%, "
                          f"IQR: [{q25_ape:.2f}%, {q75_ape:.2f}%]")

                    # Store data for plotting
                    for slice_idx, ape_val in enumerate(ape_values):
                        ape_data.append({
                            'Method': method_name,
                            'Parameter': param_name,
                            'Parameter_Index': param_idx,
                            'Slice': slice_idx,
                            'APE': ape_val
                        })

            except Exception as e:
                print(f"    ERROR processing parameter {param_idx}: {str(e)}")
                ape_results[method_name][param_idx] = np.array([])

    return ape_results, pd.DataFrame(ape_data), param_names, param_units

def perform_overall_anova_analysis_tbmf_focused(ape_df, save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Perform overall ANOVA analysis across all methods and parameters combined,
    but only display TBMF comparisons in the checkpoints
    """
    print("\n" + "=" * 80)
    print("OVERALL ANOVA ANALYSIS - TBMF vs ALL OTHER METHODS")
    print("=" * 80)

    # Prepare data for overall ANOVA
    methods = ape_df['Method'].unique()
    all_ape_values = []
    all_method_labels = []

    # Collect all APE values for each method
    method_data = {}
    for method in methods:
        method_ape = ape_df[ape_df['Method'] == method]['APE'].values
        method_data[method] = method_ape
        all_ape_values.extend(method_ape)
        all_method_labels.extend([method] * len(method_ape))

    # Perform overall one-way ANOVA
    groups = [method_data[method] for method in methods]
    f_stat, p_value = f_oneway(*groups)

    # Determine significance level
    if p_value < 0.0001:
        significance = "****"
        p_display = "p < 0.0001"
    elif p_value < 0.001:
        significance = "***"
        p_display = f"p = {p_value:.4f}"
    elif p_value < 0.01:
        significance = "**"
        p_display = f"p = {p_value:.4f}"
    elif p_value < 0.05:
        significance = "*"
        p_display = f"p = {p_value:.4f}"
    else:
        significance = "ns"
        p_display = f"p = {p_value:.4f}"

    print(f"Overall One-way ANOVA: F({len(methods)-1}, {len(all_ape_values)-len(methods)}) = {f_stat:.2f}, {p_display}")
    print(f"Significance: {significance}")

    # Perform Tukey's HSD test if significant, but only show TBMF checkpoints
    tukey_results_overall = None
    tbmf_comparisons = []

    if p_value < 0.05:
        # Perform Tukey's HSD test
        tukey_results_overall = pairwise_tukeyhsd(endog=all_ape_values, groups=all_method_labels, alpha=0.05)

        # Parse checkpoints and extract only TBMF comparisons
        for i in range(len(tukey_results_overall.summary().data) ):
            if i == 0:  # Skip header row
                continue
            row = tukey_results_overall.summary().data[i]
            group1, group2, meandiff, p_adj, lower, upper, reject = row

            # Only process TBMF-related comparisons
            if 'TBMF' in group1 or 'TBMF' in group2:
                # Format p-value display
                if p_adj < 0.0001:
                    p_display = "< 0.0001"
                    sig_marker = "****"
                elif p_adj < 0.001:
                    p_display = f"{p_adj:.4f}"
                    sig_marker = "***"
                elif p_adj < 0.01:
                    p_display = f"{p_adj:.4f}"
                    sig_marker = "**"
                elif p_adj < 0.05:
                    p_display = f"{p_adj:.4f}"
                    sig_marker = "*"
                else:
                    p_display = f"{p_adj:.4f}"
                    sig_marker = "ns"

                tbmf_comparisons.append({
                    'Group1': group1,
                    'Group2': group2,
                    'Mean_Diff': meandiff,
                    'P_adj': p_adj,
                    'Significant': reject,
                    'Significance_Level': sig_marker,
                    'P_display': p_display
                })

        # Display TBMF comparisons only
        print(f"\n" + "=" * 60)
        print("TBMF DISTINCTNESS ANALYSIS")
        print("=" * 60)

        tbmf_mean = np.mean(method_data['TBMF'])
        tbmf_median = np.median(method_data['TBMF'])
        print(f"TBMF Mean APE across all parameters: {tbmf_mean:.2f}%")
        print(f"TBMF Median APE across all parameters: {tbmf_median:.2f}%")

        print(f"\nTBMF vs Other Methods - Tukey's HSD Results:")
        print(f"{'Method':<12} {'Mean APE':<12} {'Median APE':<12} {'Mean Diff':<15} {'p-value':<12} {'Significance':<12}")
        print("-" * 85)

        for comp in tbmf_comparisons:
            other_method = comp['Group1'] if comp['Group1'] != 'TBMF' else comp['Group2']
            other_mean = np.mean(method_data[other_method])
            other_median = np.median(method_data[other_method])
            diff = other_mean - tbmf_mean

            print(f"{other_method:<12} {other_mean:<12.2f} {other_median:<12.2f} {diff:+.2f}%        {comp['P_display']:<12} {comp['Significance_Level']:<12}")

    return {
        'f_statistic': f_stat,
        'p_value': p_value,
        'significance': significance,
        'tukey_results': tukey_results_overall,
        'tbmf_comparisons': tbmf_comparisons,
        'method_means': {method: np.mean(data) for method, data in method_data.items()},
        'method_medians': {method: np.median(data) for method, data in method_data.items()}
    }

def create_overall_comparison_plot_with_anova_fixed(ape_df, overall_anova_results,
                                                   save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Create the overall comparison plot with ANOVA checkpoints and individual points,
    highlighting TBMF's distinctness - SORTED BY MEDIAN APE
    """
    fig = plt.figure(figsize=(16, 10), facecolor='white')
    ax = fig.add_subplot(111)
    ax.set_facecolor('white')

    # Define colors for each method - highlight TBMF
    method_colors = {
        'TBMF': '#FF0000',      # Red - highlighted
        'DOT_6': '#ff7f0e',     # Orange
        'DOT_30': '#d62728',    # Dark Red
        'NBMF_6': '#2ca02c',    # Green
        'NBMF_30': '#9467bd',   # Purple
        'VBMF_6': '#8c564b',    # Brown
        'VBMF_30': '#e377c2'    # Pink
    }

    # Define colors for each parameter
    param_colors = {
        'K_sw': '#1f77b4',     # Blue
        'f_s': '#ff7f0e',       # Orange
        'B_0': '#2ca02c',       # Green
        'B_1': '#d62728',       # Red
        'T_1': '#9467bd',       # Purple
        'T_2': '#8c564b'        # Brown
    }

    # Calculate median APE for each method and sort in ASCENDING order (lowest median first)
    method_medians = []
    print("\nMethod median APE calculations:")
    for method in ape_df['Method'].unique():
        method_data = ape_df[ape_df['Method'] == method]['APE']
        median_ape = np.median(method_data)
        mean_ape = np.mean(method_data)
        method_medians.append((method, median_ape))
        print(f"  {method}: Median APE = {median_ape:.2f}%, Mean APE = {mean_ape:.2f}%")

    # Sort by median APE in ascending order (lowest first - best performance on left)
    method_medians.sort(key=lambda x: x[1])
    methods_present = [method for method, _ in method_medians]

    print(f"\nMethods ordered by ascending median APE (best to worst): {methods_present}")
    print(f"Corresponding median APEs: {[f'{median:.2f}%' for _, median in method_medians]}")

    colors_for_plot = [method_colors[method] for method in methods_present]

    # Create boxplot with enhanced visibility for TBMF
    bp = ax.boxplot([ape_df[ape_df['Method'] == method]['APE'].values
                     for method in methods_present],
                    tick_labels=methods_present,
                    patch_artist=True,
                    showfliers=False,
                    medianprops={'color': 'black', 'linewidth': 2.5},
                    boxprops={'color': 'black', 'linewidth': 1.5},
                    whiskerprops={'color': 'black', 'linewidth': 1.5},
                    capprops={'color': 'black', 'linewidth': 1.5})

    # Make all boxes transparent to highlight individual points
    for i, (patch, color, method) in enumerate(zip(bp['boxes'], colors_for_plot, methods_present)):
        if method == 'TBMF':
            patch.set_facecolor("none")
            patch.set_edgecolor("black")
            patch.set_linewidth(2)  # Thicker border for TBMF
        else:
            patch.set_facecolor("none")
            patch.set_edgecolor("black")
            patch.set_linewidth(1.5)

    # Add individual data points colored by parameter
    for i, method in enumerate(methods_present):
        method_data = ape_df[ape_df['Method'] == method]

        for param in method_data['Parameter'].unique():
            param_data = method_data[method_data['Parameter'] == param]
            x_coords = np.random.normal(i + 1, 0.08, len(param_data))

            # Special styling for TBMF points
            if method == 'TBMF':
                ax.scatter(x_coords, param_data['APE'].values,
                           color=param_colors[param],
                           alpha=0.9,
                           s=60,  # Larger points for TBMF
                           edgecolors='black',
                           linewidth=1,
                           label=param if i == 0 else "",
                           zorder=4,
                           marker='o')
            else:
                ax.scatter(x_coords, param_data['APE'].values,
                           color=param_colors[param],
                           alpha=0.7,
                           s=45,
                           edgecolors='black',
                           linewidth=0.5,
                           label=param if i == 0 else "",
                           zorder=3)

    # Add ANOVA checkpoints to title
    anova_title = f"Overall ANOVA: F = {overall_anova_results['f_statistic']:.2f}, "
    if overall_anova_results['p_value'] < 0.0001:
        anova_title += f"{overall_anova_results['significance']} (p < 0.0001)"
    else:
        anova_title += f"{overall_anova_results['significance']} (p = {overall_anova_results['p_value']:.4f})"

    ax.set_title(f'Overall APE Comparison',
                fontsize=14, fontweight='bold', color='black')
    ax.set_ylabel('APE (%) - Slice Average', fontsize=12, color='black')
    ax.set_xlabel('Method', fontsize=12, color='black')
    ax.grid(True, alpha=0.1, color='gray')
    ax.tick_params(axis='x', rotation=45, colors='black')
    ax.tick_params(axis='y', colors='black')

    # Style the axes
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)

    # Set reasonable y-axis limits
    y_max = np.percentile(ape_df['APE'], 95)
    ax.set_ylim(0, max(y_max * 1.5, np.max(ape_df['APE']) * 1.1))

    # Add significance brackets for TBMF comparisons - FIXED positioning
    if overall_anova_results['significance'] != 'ns' and overall_anova_results['tbmf_comparisons']:
        y_max_plot = ax.get_ylim()[1]
        bracket_height = y_max_plot * 0.02

        # Find TBMF position in the sorted list
        tbmf_pos = methods_present.index('TBMF') + 1
        print(f"TBMF position in plot: {tbmf_pos}")

        # Add brackets for significant TBMF comparisons
        significant_tbmf_comps = [
            comp for comp in overall_anova_results['tbmf_comparisons']
            if comp['Significant'] and comp['Significance_Level'] in ['****', '***', '**', '*']
        ]

        # Sort by significance level for better display
        sig_order = {'****': 4, '***': 3, '**': 2, '*': 1, 'ns': 0}
        significant_tbmf_comps.sort(key=lambda x: sig_order[x['Significance_Level']], reverse=True)
        print(significant_tbmf_comps)

        for i, comp in enumerate(significant_tbmf_comps):
            other_method = comp['Group1'] if comp['Group1'] != 'TBMF' else comp['Group2']
            if other_method in methods_present:
                other_pos = methods_present.index(other_method) + 1
                y = y_max_plot * (0.72 + i * 0.06)  # Better spacing

                # Draw bracket
                ax.plot([min(tbmf_pos, other_pos), min(tbmf_pos, other_pos),
                        max(tbmf_pos, other_pos), max(tbmf_pos, other_pos)],
                       [y, y + bracket_height, y + bracket_height, y],
                       'k-', linewidth=1.5, alpha=0.8)

                # Add significance text
                    # Add significance text - show exact p-value if > 0.001
                if comp['P_adj'] > 0.001:
                        # Show exact p-value for p > 0.001 (includes *, **, and ns)
                        display_text = f"p = {comp['P_adj']:.3f}"
                else:
                        # Show asterisks for p ≤ 0.001 (*** and ****)
                        display_text = comp['Significance_Level']

                ax.text((tbmf_pos + other_pos) / 2, y + y_max_plot * 0.025,
                           display_text,
                           ha='center', va='bottom', fontweight='bold',
                           color='red', fontsize=9)

    # Add legend for parameters
    legend_elements = [plt.scatter([], [], color=param_colors[param], s=60, alpha=0.7,
                                 edgecolors='black', linewidth=0.5, label=param)
                      for param in sorted(param_colors.keys())]
    legend = ax.legend(handles=legend_elements,
                      loc='upper right', frameon=False, fancybox=True, shadow=True)
    legend.get_title().set_color('black')
    legend.get_title().set_fontweight('bold')
    for text in legend.get_texts():
        text.set_color('black')

    plt.tight_layout()

    # Save the plot
    plot_filename = f"{save_path}overall_anova_comparison_with_tbmf_highlight_median_sorted.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

    print(f"Overall ANOVA comparison plot (sorted by median) with TBMF highlighting saved to: {plot_filename}")

def create_ape_summary_table_slice_based(ape_df):
    """
    Create a summary table with slice-based APE statistics
    """
    summary_stats = []

    for param_name in ape_df['Parameter'].unique():
        param_data = ape_df[ape_df['Parameter'] == param_name]

        for method in param_data['Method'].unique():
            method_data = param_data[param_data['Method'] == method]['APE']

            if len(method_data) > 0:
                summary_stats.append({
                    'Parameter': param_name,
                    'Method': method,
                    'N_Slices': len(method_data),
                    'Mean_APE': np.mean(method_data),
                    'Median_APE': np.median(method_data),
                    'Q25_APE': np.percentile(method_data, 25),
                    'Q75_APE': np.percentile(method_data, 75),
                    'Min_APE': np.min(method_data),
                    'Max_APE': np.max(method_data),
                    'Std_APE': np.std(method_data)
                })

    return pd.DataFrame(summary_stats)

def run_tbmf_focused_anova_analysis():
    """
    Main function to run TBMF-focused ANOVA analysis with median-based sorting
    """
    # Calculate slice-based APE for all methods and parameters
    ape_results, ape_df, param_names, param_units = calculate_all_ape_phantom_slice_based(
        gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
        y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
        y_alex_vbmf_6_masked, y_alex_vbmf_30_masked
    )

    # Perform overall ANOVA analysis focusing on TBMF
    overall_anova_results = perform_overall_anova_analysis_tbmf_focused(ape_df)

    # Create overall comparison plot with median-based sorting
    create_overall_comparison_plot_with_anova_fixed(ape_df, overall_anova_results)

    # Create summary table
    summary_table = create_ape_summary_table_slice_based(ape_df)

    # Display summary statistics
    print("\n" + "=" * 80)
    print("SLICE-BASED APE SUMMARY STATISTICS")
    print("=" * 80)

    # Show summary for each parameter
    for param in param_names[:6]:
        param_summary = summary_table[summary_table['Parameter'] == param]
        if not param_summary.empty:
            print(f"\n{param}:")
            print("-" * 40)
            for _, row in param_summary.iterrows():
                print(f"  {row['Method']:10}: "
                      f"Median={row['Median_APE']:6.2f}%, "
                      f"Mean={row['Mean_APE']:6.2f}%, "
                      f"IQR=[{row['Q25_APE']:5.2f}%, {row['Q75_APE']:5.2f}%], "
                      f"N_slices={row['N_Slices']:2d}")

    # Save detailed checkpoints
    save_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"

    # Save comprehensive APE data
    ape_csv_filename = f"{save_path}ape_slice_based_comprehensive_data_tbmf_focused_median_sorted.csv"
    ape_df.to_csv(ape_csv_filename, index=False)

    # Save summary statistics
    summary_csv_filename = f"{save_path}ape_slice_based_summary_statistics_tbmf_focused_median_sorted.csv"
    summary_table.to_csv(summary_csv_filename, index=False)

    # Save ANOVA checkpoints
    overall_results_df = pd.DataFrame([{
        'Analysis': 'Overall_ANOVA',
        'F_statistic': overall_anova_results['f_statistic'],
        'P_value': overall_anova_results['p_value'],
        'Significance': overall_anova_results['significance']
    }])

    if overall_anova_results['tbmf_comparisons']:
        tbmf_results_df = pd.DataFrame(overall_anova_results['tbmf_comparisons'])
        anova_results_filename = f"{save_path}tbmf_anova_results_median_sorted.csv"
        with open(anova_results_filename, 'w') as f:
            f.write("Overall ANOVA Results\n")
            overall_results_df.to_csv(f, index=False)
            f.write("\nTBMF Comparisons\n")
            tbmf_results_df.to_csv(f, index=False)

    print(f"\nResults saved:")
    print(f"- Comprehensive slice-based APE data: '{ape_csv_filename}'")
    print(f"- Summary statistics: '{summary_csv_filename}'")
    if overall_anova_results['tbmf_comparisons']:
        print(f"- TBMF ANOVA checkpoints: '{anova_results_filename}'")

    return ape_results, ape_df, summary_table, overall_anova_results

# Run the complete TBMF-focused analysis with median-based sorting
if __name__ == "__main__":
    results = run_tbmf_focused_anova_analysis()

In [ ]:
def perform_parameter_wise_anova_analysis_tbmf_focused(ape_df, save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Perform ANOVA analysis for each parameter separately, focusing on TBMF comparisons
    """
    print("\n" + "=" * 80)
    print("PARAMETER-WISE ANOVA ANALYSIS - TBMF vs OTHER METHODS")
    print("=" * 80)

    param_anova_results = {}

    # Get unique parameters
    parameters = ape_df['Parameter'].unique()

    for param in parameters:
        print(f"\n{'-' * 60}")
        print(f"PARAMETER: {param}")
        print(f"{'-' * 60}")

        # Filter data for this parameter
        param_data = ape_df[ape_df['Parameter'] == param]

        if param_data.empty:
            print(f"No data available for parameter {param}")
            continue

        # Get methods available for this parameter
        methods = param_data['Method'].unique()

        if len(methods) < 2:
            print(f"Not enough methods for comparison (only {len(methods)} methods available)")
            continue

        # Prepare data for ANOVA
        method_data = {}
        for method in methods:
            method_ape = param_data[param_data['Method'] == method]['APE'].values
            method_data[method] = method_ape

        # Perform one-way ANOVA for this parameter
        groups = [method_data[method] for method in methods]
        f_stat, p_value = f_oneway(*groups)

        # Determine significance level
        if p_value < 0.0001:
            significance = "****"
            p_display = "p < 0.0001"
        elif p_value < 0.001:
            significance = "***"
            p_display = f"p = {p_value:.4f}"
        elif p_value < 0.01:
            significance = "**"
            p_display = f"p = {p_value:.4f}"
        elif p_value < 0.05:
            significance = "*"
            p_display = f"p = {p_value:.4f}"
        else:
            significance = "ns"
            p_display = f"p = {p_value:.4f}"

        print(f"One-way ANOVA for {param}: F({len(methods)-1}, {len(param_data)-len(methods)}) = {f_stat:.2f}, {p_display}")
        print(f"Significance: {significance}")

        # Initialize parameter checkpoints
        param_anova_results[param] = {
            'f_statistic': f_stat,
            'p_value': p_value,
            'significance': significance,
            'methods': list(methods),
            'tbmf_comparisons': [],
            'method_means': {method: np.mean(data) for method, data in method_data.items()},
            'method_medians': {method: np.median(data) for method, data in method_data.items()}
        }

        # Perform Tukey's HSD test if significant and TBMF is present
        if p_value < 0.05 and 'TBMF' in methods:
            # Prepare data for Tukey's test
            all_ape_values = []
            all_method_labels = []
            for method in methods:
                method_ape = param_data[param_data['Method'] == method]['APE'].values
                all_ape_values.extend(method_ape)
                all_method_labels.extend([method] * len(method_ape))

            # Perform Tukey's HSD test
            tukey_results = pairwise_tukeyhsd(endog=all_ape_values, groups=all_method_labels, alpha=0.05)
            print(f"Full Tukey checkpoints for {param}:")
            print(tukey_results.summary())
            print(f"Number of comparisons: {len(tukey_results.summary().data)}")
            for i, row in enumerate(tukey_results.summary().data):
                print(f"Row {i}: {row}")
            param_anova_results[param]['tukey_results'] = tukey_results

            # Parse checkpoints and extract only TBMF comparisons
            tbmf_comparisons = []
            for i in range(len(tukey_results.summary().data) ):
                if i == 0:  # Skip header row
                    continue
                row = tukey_results.summary().data[i]
                group1, group2, meandiff, p_adj, lower, upper, reject = row

                # Only process TBMF-related comparisons
                if 'TBMF' in group1 or 'TBMF' in group2:
                    # Format p-value display
                    if p_adj < 0.0001:
                        p_display = "< 0.0001"
                        sig_marker = "****"
                    elif p_adj < 0.001:
                        p_display = f"{p_adj:.4f}"
                        sig_marker = "***"
                    elif p_adj < 0.01:
                        p_display = f"{p_adj:.4f}"
                        sig_marker = "**"
                    elif p_adj < 0.05:
                        p_display = f"{p_adj:.4f}"
                        sig_marker = "*"
                    else:
                        p_display = f"{p_adj:.4f}"
                        sig_marker = "ns"

                    tbmf_comparisons.append({
                        'Group1': group1,
                        'Group2': group2,
                        'Mean_Diff': meandiff,
                        'P_adj': p_adj,
                        'Significant': reject,
                        'Significance_Level': sig_marker,
                        'P_display': p_display
                    })

            param_anova_results[param]['tbmf_comparisons'] = tbmf_comparisons

            # Display TBMF comparisons for this parameter
            if 'TBMF' in method_data:
                tbmf_mean = np.mean(method_data['TBMF'])
                tbmf_median = np.median(method_data['TBMF'])
                print(f"\nTBMF Performance for {param}:")
                print(f"  Mean APE: {tbmf_mean:.2f}%")
                print(f"  Median APE: {tbmf_median:.2f}%")

                if tbmf_comparisons:
                    print(f"\nTBMF vs Other Methods for {param} - Tukey's HSD Results:")
                    print(f"{'Method':<12} {'Mean APE':<12} {'Median APE':<12} {'Mean Diff':<15} {'p-value':<12} {'Significance':<12}")
                    print("-" * 85)

                    for comp in tbmf_comparisons:
                        other_method = comp['Group1'] if comp['Group1'] != 'TBMF' else comp['Group2']
                        other_mean = np.mean(method_data[other_method])
                        other_median = np.median(method_data[other_method])
                        diff = other_mean - tbmf_mean

                        print(f"{other_method:<12} {other_mean:<12.2f} {other_median:<12.2f} {diff:+.2f}%        {comp['P_display']:<12} {comp['Significance_Level']:<12}")
        else:
            param_anova_results[param]['tukey_results'] = None

    return param_anova_results

def create_enhanced_parameter_wise_plots(ape_df, param_anova_results,
                                       save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Create enhanced parameter-wise comparison plots with better visibility and significance annotations
    """
    # Define colors for each method - highlight TBMF
    method_colors = {
        'TBMF': '#0000FF',      # Blue
        'DOT_6': '#ff7f0e',     # Orange
        'DOT_30': '#d62728',    # Dark Red
        'NBMF_6': '#2ca02c',    # Green
        'NBMF_30': '#9467bd',   # Purple
        'VBMF_6': '#8c564b',    # Brown
        'VBMF_30': '#e377c2'    # Pink
    }

    parameters = ape_df['Parameter'].unique()

    # Create figure with better spacing
    fig = plt.figure(figsize=(20, 15), facecolor='white')

    # Define subplot layout - 2 rows, 3 columns
    n_cols = 3
    n_rows = 2
    y_lims = [100,100,200,20,20,100]

    for i, param in enumerate(parameters):

        ax = plt.subplot(n_rows, n_cols, i+1)
        ax.set_facecolor('white')


        # Filter data for this parameter
        param_data = ape_df[ape_df['Parameter'] == param]

        if param_data.empty:
            ax.text(0.5, 0.5, f'No data for {param}', ha='center', va='center',
                   transform=ax.transAxes, fontsize=12)
            ax.set_title(param, fontsize=14, fontweight='bold')
            continue

        # Get methods and sort by median APE
        methods = param_data['Method'].unique()
        method_medians = []
        for method in methods:
            method_ape = param_data[param_data['Method'] == method]['APE']
            median_ape = np.median(method_ape)
            method_medians.append((method, median_ape))

        # Sort by median APE in ascending order
        method_medians.sort(key=lambda x: x[1])
        methods_sorted = [method for method, _ in method_medians]

        # Create enhanced boxplot
        bp = ax.boxplot([param_data[param_data['Method'] == method]['APE'].values
                        for method in methods_sorted],
                       tick_labels=methods_sorted,
                       patch_artist=True,
                       showfliers=False,
                       medianprops={'color': 'black', 'linewidth': 2.5},
                       boxprops={'linewidth': 1.5},
                       whiskerprops={'linewidth': 1.5},
                       capprops={'linewidth': 1.5})

        # Color the boxes with special emphasis on TBMF
        for j, (patch, method) in enumerate(zip(bp['boxes'], methods_sorted)):
            color = method_colors.get(method, '#808080')
            if method == 'TBMF':
                patch.set_facecolor("none")
                patch.set_edgecolor(color)
                patch.set_linewidth(2)
            else:
                patch.set_facecolor("none")
                patch.set_edgecolor(color)
                patch.set_linewidth(1.5)

        # Add individual points with jitter
        for j, method in enumerate(methods_sorted):
            method_ape = param_data[param_data['Method'] == method]['APE'].values
            x_coords = np.random.normal(j + 1, 0.05, len(method_ape))

            if method == 'TBMF':
                ax.scatter(x_coords, method_ape, color=method_colors[method],
                          alpha=0.7, s=55, edgecolors='black', linewidth=1.2,
                          zorder=4, marker='o')
            else:
                ax.scatter(x_coords, method_ape, color=method_colors.get(method, '#808080'),
                          alpha=0.65, s=45, edgecolors='black', linewidth=1,
                          zorder=3)

        # Enhanced title with ANOVA checkpoints and effect size
        if param in param_anova_results:
            anova_result = param_anova_results[param]
            if anova_result['significance'] != 'ns':
                title_text = f"{param}\n{anova_result['significance']}"
                if anova_result['p_value'] < 0.0001:
                    title_text += " (p < 0.0001)"
                else:
                    title_text += f" (p = {anova_result['p_value']:.4f})"
            else:
                title_text = f"{param}\nns (p = {anova_result['p_value']:.3f})"
        else:
            title_text = param

        ax.set_title(title_text, fontsize=12, fontweight='bold', pad=15)

        # Enhanced axis labels and formatting
        ax.set_ylabel('APE (%)', fontsize=11, fontweight='bold')
        ax.set_xlabel('Method (ordered by median APE)', fontsize=10)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.tick_params(axis='x', rotation=45, labelsize=9)
        ax.tick_params(axis='y', labelsize=9)

        # Set reasonable y-axis limits
        all_values = param_data['APE'].values
        y_max = np.percentile(all_values, 95)
        ax.set_ylim(0,y_lims[i])

        # Add significance brackets for TBMF comparisons if significant
        if (param in param_anova_results and
            param_anova_results[param]['significance'] != 'ns' and
            'TBMF' in methods_sorted and
            param_anova_results[param]['tbmf_comparisons']):

            tbmf_pos = methods_sorted.index('TBMF') + 1
            significant_comps = [comp for comp in param_anova_results[param]['tbmf_comparisons']
                               if comp['Significant']]

            y_max_plot = ax.get_ylim()[1]

            for k, comp in enumerate(significant_comps):
                other_method = comp['Group1'] if comp['Group1'] != 'TBMF' else comp['Group2']
                if other_method in methods_sorted:
                    other_pos = methods_sorted.index(other_method) + 1
                    y = y_max_plot * (0.8 + k * 0.03)

                    # Draw bracket
                    ax.plot([min(tbmf_pos, other_pos), min(tbmf_pos, other_pos),
                            max(tbmf_pos, other_pos), max(tbmf_pos, other_pos)],
                           [y, y + y_max_plot * 0.02, y + y_max_plot * 0.02, y],
                           'k-', linewidth=1.2, alpha=0.7)

                    # Add significance text - show exact p-value if > 0.001
                    if comp['P_adj'] > 0.001:
                        # Show exact p-value for p > 0.001 (includes *, **, and ns)
                        display_text = f"p = {comp['P_adj']:.3f}"
                    else:
                        # Show asterisks for p ≤ 0.001 (*** and ****)
                        display_text = comp['Significance_Level']

                    ax.text((tbmf_pos + other_pos) / 2, y + y_max_plot * 0.026,
                           display_text,
                           ha='center', va='bottom', fontweight='bold',
                           color='red', fontsize=9)

        # Style the axes
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_color('black')
            spine.set_linewidth(1)

    plt.tight_layout(pad=3.0)

    # Save the enhanced plot
    plot_filename = f"{save_path}parameter_wise_anova_comparison_tbmf.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

    print(f"Enhanced parameter-wise ANOVA comparison plots saved to: {plot_filename}")

def save_parameter_anova_results(param_anova_results, save_path="/home/sahar/Models/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"):
    """
    Save parameter-wise ANOVA checkpoints to CSV files
    """
    # Create summary of all parameter ANOVA checkpoints
    param_summary = []
    tbmf_comparisons_all = []

    for param, results in param_anova_results.items():
        param_summary.append({
            'Parameter': param,
            'F_statistic': results['f_statistic'],
            'P_value': results['p_value'],
            'Significance': results['significance'],
            'N_methods': len(results['methods']),
            'Methods': ', '.join(results['methods']),
            'TBMF_mean_APE': results['method_means'].get('TBMF', 'N/A'),
            'TBMF_median_APE': results['method_medians'].get('TBMF', 'N/A')
        })

        # Collect TBMF comparisons
        for comp in results['tbmf_comparisons']:
            comp_copy = comp.copy()
            comp_copy['Parameter'] = param
            tbmf_comparisons_all.append(comp_copy)

    # Save parameter summary
    param_summary_df = pd.DataFrame(param_summary)
    param_summary_filename = f"{save_path}parameter_wise_anova_summary.csv"
    param_summary_df.to_csv(param_summary_filename, index=False)

    # Save TBMF comparisons
    if tbmf_comparisons_all:
        tbmf_comparisons_df = pd.DataFrame(tbmf_comparisons_all)
        tbmf_comparisons_filename = f"{save_path}parameter_wise_tbmf_comparisons.csv"
        tbmf_comparisons_df.to_csv(tbmf_comparisons_filename, index=False)

        print(f"Parameter-wise TBMF comparisons saved to: {tbmf_comparisons_filename}")

    print(f"Parameter-wise ANOVA summary saved to: {param_summary_filename}")

    return param_summary_df, pd.DataFrame(tbmf_comparisons_all) if tbmf_comparisons_all else None

def run_comprehensive_tbmf_focused_anova_analysis():
    """
    Main function to run comprehensive TBMF-focused ANOVA analysis (both overall and parameter-wise)
    """
    # Calculate slice-based APE for all methods and parameters
    ape_results, ape_df, param_names, param_units = calculate_all_ape_phantom_slice_based(
        gt, tbmf_selective, y_nikita_6_masked, y_nikita_30_masked,
        y_alex_nbmf_6_masked, y_alex_nbmf_30_masked,
        y_alex_vbmf_6_masked, y_alex_vbmf_30_masked
    )

    # Perform overall ANOVA analysis focusing on TBMF
    print("PERFORMING OVERALL ANOVA ANALYSIS...")
    overall_anova_results = perform_overall_anova_analysis_tbmf_focused(ape_df)

    # Perform parameter-wise ANOVA analysis focusing on TBMF
    print("\nPERFORMING PARAMETER-WISE ANOVA ANALYSIS...")
    param_anova_results = perform_parameter_wise_anova_analysis_tbmf_focused(ape_df)

    # Create overall comparison plot
    print("\nCREATING OVERALL COMPARISON PLOT...")
    create_overall_comparison_plot_with_anova_fixed(ape_df, overall_anova_results)

    # Create parameter-wise comparison plots
    print("\nCREATING PARAMETER-WISE COMPARISON PLOTS...")
    create_enhanced_parameter_wise_plots(ape_df, param_anova_results)

    # Create summary table
    summary_table = create_ape_summary_table_slice_based(ape_df)

    # Save parameter-wise ANOVA checkpoints
    print("\nSAVING PARAMETER-WISE RESULTS...")
    param_summary_df, tbmf_comparisons_df = save_parameter_anova_results(param_anova_results)

    # Display summary statistics
    print("\n" + "=" * 80)
    print("COMPREHENSIVE SLICE-BASED APE SUMMARY STATISTICS")
    print("=" * 80)

    # Show summary for each parameter
    for param in param_names[:6]:
        param_summary = summary_table[summary_table['Parameter'] == param]
        if not param_summary.empty:
            print(f"\n{param}:")
            print("-" * 40)
            for _, row in param_summary.iterrows():
                print(f"  {row['Method']:10}: "
                      f"Median={row['Median_APE']:6.2f}%, "
                      f"Mean={row['Mean_APE']:6.2f}%, "
                      f"IQR=[{row['Q25_APE']:5.2f}%, {row['Q75_APE']:5.2f}%], "
                      f"N_slices={row['N_Slices']:2d}")

            # Show ANOVA checkpoints for this parameter
            if param in param_anova_results:
                anova_result = param_anova_results[param]
                print(f"    ANOVA: F = {anova_result['f_statistic']:.2f}, {anova_result['significance']}")

    # Save comprehensive checkpoints
    save_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"

    # Save comprehensive APE data
    ape_csv_filename = f"{save_path}comprehensive_ape_slice_based_data_with_parameter_anova.csv"
    ape_df.to_csv(ape_csv_filename, index=False)

    # Save summary statistics
    summary_csv_filename = f"{save_path}comprehensive_ape_summary_statistics_with_parameter_anova.csv"
    summary_table.to_csv(summary_csv_filename, index=False)

    print(f"\nCOMPREHENSIVE RESULTS SAVED:")
    print(f"- Comprehensive APE data: '{ape_csv_filename}'")
    print(f"- Summary statistics: '{summary_csv_filename}'")

    return {
        'ape_results': ape_results,
        'ape_df': ape_df,
        'summary_table': summary_table,
        'overall_anova_results': overall_anova_results,
        'param_anova_results': param_anova_results,
        'param_summary_df': param_summary_df,
        'tbmf_comparisons_df': tbmf_comparisons_df
    }

# Update the main execution
if __name__ == "__main__":
    # Run the comprehensive analysis (both overall and parameter-wise)
    comprehensive_results = run_comprehensive_tbmf_focused_anova_analysis()

# ICC and image creation for comperisson of ph,fs, ksw and mM

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

ph_gt_tmbf_path = "/comp/pH_maps_3D_test/scan12.h5"
mM_gt_tmbf_path = "/comp/mM_maps_3D_test/scan12.h5"
ph_gt = h5py.File(ph_gt_tmbf_path)
mM_gt = h5py.File(mM_gt_tmbf_path)
keys_ph = list(ph_gt.keys())
keys_mM = list(mM_gt.keys())
ph_gt = ph_gt[keys_ph[-1]][:]
mM_gt = mM_gt[keys_mM[-1]][:]
gt_ph_mm = np.stack([ph_gt, mM_gt], axis=0)
gt_ph_mm[gt_ph_mm < 2] = np.nan

y_pred_path= "/comp/y_pred_4D_pH_mM_scaled.h5"
y_pred = h5py.File(y_pred_path)
y_pred_keys = list(y_pred.keys())
y_pred = y_pred[y_pred_keys[-1]][:]
y_pred = np.transpose(y_pred, (1,0, 2,3))
y_pred[y_pred <2] = np.nan
tbmf_ksw_fs = tbmf[:2]

# Let's see what we're working with - show original data before any transformation
fig, axes = plt.subplots(4, 2, figsize=(10, 5))

# Store the image objects from imshow() calls
im1 = axes[0,0].imshow(tbmf_ksw_fs[0,10], cmap="viridis")
axes[0,0].set_title('ksw Original')
axes[0,0].axis("off")

im2 = axes[0,1].imshow(tbmf_ksw_fs[1,10], cmap="viridis")
axes[0,1].set_title('fs Original')
axes[0,1].axis("off")

im3 = axes[1,0].imshow(y_pred[0,10], cmap="viridis")
axes[1,0].set_title('y_pred[0] (pH)')
axes[1,0].axis("off")

im4 = axes[1,1].imshow(y_pred[1,10], cmap="viridis")
axes[1,1].set_title('y_pred[1] (mM)')
axes[1,1].axis("off")


# Gt
# Store the image objects from imshow() calls
im5 = axes[2,0].imshow(tbmf_gt[0,10], cmap="viridis")
axes[2,0].set_title('ksw gt')
axes[2,0].axis("off")

im6 = axes[2,1].imshow(tbmf_gt[1,10], cmap="viridis")
axes[2,1].set_title('fs gt')
axes[2,1].axis("off")

im7 = axes[3,0].imshow(gt_ph_mm[0,10], cmap="viridis")
axes[3,0].set_title('GT (pH)')
axes[3,0].axis("off")

im8 = axes[3,1].imshow(gt_ph_mm[1,10], cmap="viridis")
axes[3,1].set_title('(mM)')
axes[3,1].axis("off")


# Add colorbars
fig.colorbar(im1, ax=axes[0,0], shrink=0.8)
fig.colorbar(im2, ax=axes[0,1], shrink=0.8)
fig.colorbar(im3, ax=axes[1,0], shrink=0.8)
fig.colorbar(im4, ax=axes[1,1], shrink=0.8)
fig.colorbar(im5, ax=axes[2,0], shrink=0.8)
fig.colorbar(im6, ax=axes[2,1], shrink=0.8)
fig.colorbar(im7, ax=axes[3,0], shrink=0.8)
fig.colorbar(im8, ax=axes[3,1], shrink=0.8)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

# Set up the methods and their data - CORRECTED
methods = {
    'KSW': (tbmf_gt[0], tbmf_ksw_fs[0]),
    'pH': (gt_ph_mm[0], y_pred[0]),
    'fs': (tbmf_gt[1], tbmf_ksw_fs[1]),  # FIXED: was tbmf_ksw_fs[0], should be [1]
    'mM': (gt_ph_mm[1], y_pred[1]),
}

# Parameter names
param_names = [
    r"$K_{sw}$ (S$^{-1}$)",
    r"pH",
    r"$f_s$ (%)",
    r"mM",
]

# Create figure with black background
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.patch.set_facecolor('black')  # Set black background for figure

# Set black background for all subplots
for i in range(2):
    for j in range(4):
        axes[i, j].set_facecolor('black')

# Axis limits for each parameter [min, max]
axis_limits = {
    0: [0, 1400],      # K_ssw
    1: [3.5, 7],       # pH - CORRECTED range
    2: [0, 120],       # f_s
    3: [0, 120],       # mM
}

results_summary = []

# Process each method - CORRECTED loop structure
for method_idx, (method_name, (data1, data2)) in enumerate(methods.items()):
    print(f"\nProcessing {method_name}...")

    # Row 0: Statistical analysis
    if method_idx < 4:  # We have 4 methods
        ax = axes[0, method_idx]

        # Extract parameter data
        param1 = data1.copy()  # Ground truth
        param2 = data2.copy()  # Prediction

        print(f"\n{method_name}:")
        print(f"  param1 range: [{np.nanmin(param1):.3f}, {np.nanmax(param1):.3f}]")
        print(f"  param2 range: [{np.nanmin(param2):.3f}, {np.nanmax(param2):.3f}]")
        print(f"  axis limits: {axis_limits[method_idx]}")
        print(f"  Valid points: {np.sum(~np.isnan(param1) & ~np.isnan(param2))}")

        # Calculate ICC
        try:
            icc_value = slicewise_icc(param1, param2, icctype='ICC2')
            print(f"ICC for {method_name}: {icc_value:.3f}")
        except:
            icc_value = np.nan

        # Calculate Pearson correlation
        mask = ~np.isnan(param1) & ~np.isnan(param2)
        param1_clean = param1[mask]
        param2_clean = param2[mask]

        try:
            if len(param1_clean) > 10:
                pearson_r, pearson_p = stats.pearsonr(param1_clean, param2_clean)
                print(f"Pearson r for {method_name}: {pearson_r:.3f}, p={pearson_p:.3e}")

                # For plotting, subsample if too many points
                if len(param1_clean) > 5000:
                    subsample_idx = np.random.choice(len(param1_clean), 5000, replace=False)
                    param1_plot = param1_clean[subsample_idx]
                    param2_plot = param2_clean[subsample_idx]
                else:
                    param1_plot = param1_clean
                    param2_plot = param2_clean

                # Create scatter plot
                ax.scatter(param1_plot, param2_plot, alpha=0.6, s=1.1, color='steelblue')

                # Set axis limits
                lims = axis_limits[method_idx]
                ax.set_xlim(lims)
                ax.set_ylim(lims)

                # Add identity line
                ax.plot(lims, lims, 'r--', alpha=0.8, linewidth=1)

                # Add regression line
                if not (np.isnan(pearson_r) or pearson_r == 0):
                    slope, intercept = np.polyfit(param1_clean, param2_clean, 1)
                    x_reg = np.array(lims)
                    y_reg = slope * x_reg + intercept
                    y_reg = np.clip(y_reg, lims[0], lims[1])
                    ax.plot(x_reg, y_reg, 'orange', linewidth=1.5)

                # Set labels and title with white color
                ax.set_xlabel(f'Ground Truth {method_name}', color='white')
                ax.set_ylabel(f'Predicted {method_name}', color='white')
                ax.set_title(param_names[method_idx], fontweight='bold', color='white')

                # Set tick colors to white
                ax.tick_params(colors='white')

                # Set spine colors to white
                for spine in ax.spines.values():
                    spine.set_color('white')

                # Add statistics text
                stats_text = f'r = {pearson_r:.3f}\n'
                if pearson_p < 0.001:
                    stats_text += 'p < 0.001\n'
                else:
                    stats_text += f'p = {pearson_p:.3f}\n'
                stats_text += f'ICC = {icc_value:.3f}\n'

                ax.text(0.05, 0.95, stats_text, transform=ax.transAxes,
                       fontsize=10, verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

                ax.grid(True, alpha=0.3, color='white')

                # Store checkpoints
                results_summary.append({
                    'Method': method_name,
                    'Parameter': param_names[method_idx],
                    'Pearson_r': pearson_r,
                    'Pearson_p': pearson_p,
                    'ICC': icc_value,
                    'N_points': len(param1_clean)
                })
            else:
                ax.text(0.5, 0.5, 'Insufficient\ndata', transform=ax.transAxes,
                       ha='center', va='center', fontsize=10, color='white')
                ax.set_title(param_names[method_idx], fontweight='bold', color='white')

        except Exception as e:
            ax.text(0.5, 0.5, 'Error in\ncalculation', transform=ax.transAxes,
                   ha='center', va='center', fontsize=10, color='white')
            ax.set_title(param_names[method_idx], fontweight='bold', color='white')
            print(f"Error in {method_name}: {e}")

# Row 1: Example images
slice_idx_p = 10
slice_index_k = 8
images = [
    (tbmf_ksw_fs[0, slice_index_k], 'K_sw '),
    (y_pred[0, slice_idx_p], 'pH'),
    (tbmf_ksw_fs[1, slice_index_k], 'f_s'),
    (y_pred[1, slice_idx_p], 'mM')
]
cbar = create_cmaps()
cbar_list = [cbar[0], cbar[0], cbar[1], cbar[1]]
vmin = [0,3,0,0]
vmax = [1400,7,110,110]
for i, (img_data, title) in enumerate(images):
    ax = axes[1, i]

    # Create image
    im = ax.imshow(img_data, cmap=cbar_list[i], vmin=vmin[i], vmax=vmax[i])
    ax.set_title(title, fontweight='bold', color='white')
    ax.axis('off')

    # Get the position of the current subplot
    pos = ax.get_position()

    # Create colorbar axis below the image
    cax = fig.add_axes([
        pos.x0,           # Same left position as subplot
        pos.y0 - 0.08,    # Below the subplot
        pos.width,        # Same width as subplot
        0.02              # Height of colorbar
    ])

    # Set colorbar background to black
    cax.set_facecolor('black')

    # Add colorbar
    cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
    cbar.ax.tick_params(colors='white')  # White tick labels
    cbar.outline.set_edgecolor('white')  # White outline

# Adjust layout
plt.subplots_adjust(top=0.93, bottom=0.15)  # Adjusted bottom to make room for colorbars


# Uncomment these lines to save and create summary
# # Save the plot
save_path = "/Dinor_revision/new_phantom/dinor_train/code_addition/phanton_comp/"
plt.savefig(f"{save_path}/ICC_correlation_analysis_ph_mm_ksw_fs_black_bg.png",
            dpi=300, bbox_inches='tight', facecolor='black')
#
# Create summary table
results_df = pd.DataFrame(results_summary)
print("\n" + "="*80)
print("SUMMARY TABLE - pH, mM, K_sw, f_s Analysis")
print("="*80)

if not results_df.empty:
    print("\nDetailed Results:")
    print("-" * 60)
    for _, row in results_df.iterrows():
        print(f"{row['Method']:>8}: ICC={row['ICC']:.3f}, "
              f"r={row['Pearson_r']:.3f}, "
              f"p={row['Pearson_p']:}, "
              f"N={row['N_points']:,}")

    # Save checkpoints to CSV
    csv_path = f"{save_path}/correlation_icc_results_ph_mm_ksw_fs_black_bg.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"\nDetailed checkpoints saved to: {csv_path}")